# ENHANCED EVALUATION: QUESTION QUALITY & REASONING FAITHFULNESS
This section implements comprehensive evaluation metrics specifically designed for Q&A-CoT format:

## Evaluation Components

1. **Question Quality Assessment**: Evaluates the clarity, relevance, and logical progression of self-generated questions
2. **Reasoning Faithfulness**: Measures how well the answers align with factual knowledge and logical reasoning
3. **Chain-of-Thought Coherence**: Assesses the logical flow and consistency across the Q&A sequence
4. **Final Answer Accuracy**: Traditional accuracy measurement with confidence scoring

## When to Run

- After completing progressive curriculum training
- Before and after token emphasis training to measure improvement
- For model comparison between different training approaches

## Output

- Detailed evaluation metrics saved to `enhanced_evaluation_results.json`
- Question quality analysis and reasoning faithfulness scores
- Comparative analysis with baseline and previous training phases

In [ ]:
# ============================================================================
# ENHANCED EVALUATION: QUESTION QUALITY & REASONING FAITHFULNESS
# ============================================================================

import re
import numpy as np
from typing import Dict, List, Tuple, Any
from dataclasses import dataclass
from collections import defaultdict
import torch.nn.functional as F

@dataclass
class QuestionQualityMetrics:
    """Metrics for evaluating question quality in Q&A-CoT format."""
    clarity_score: float  # How clear and understandable the questions are (0-1)
    relevance_score: float  # How relevant questions are to the main problem (0-1)
    progression_score: float  # How well questions build upon each other (0-1)
    specificity_score: float  # How specific vs generic the questions are (0-1)
    num_questions: int  # Total number of questions generated
    avg_question_length: float  # Average question length in words

@dataclass 
class ReasoningFaithfulnessMetrics:
    """Metrics for evaluating reasoning faithfulness."""
    factual_consistency: float  # Consistency with known facts (0-1)
    logical_coherence: float  # Internal logical consistency (0-1)
    evidence_support: float  # How well answers are supported by evidence (0-1)
    answer_alignment: float  # How well intermediate answers lead to final answer (0-1)
    hallucination_rate: float  # Rate of factual errors or hallucinations (0-1)

@dataclass
class EnhancedEvaluationResults:
    """Complete enhanced evaluation results."""
    accuracy: float
    question_quality: QuestionQualityMetrics
    reasoning_faithfulness: ReasoningFaithfulnessMetrics
    confidence_calibration: Dict[str, float]
    format_compliance: float
    reasoning_depth: float
    total_examples: int
    
class EnhancedQACoTEvaluator:
    """Enhanced evaluator for Q&A-CoT format with quality and faithfulness metrics."""
    
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        
        # Question quality patterns
        self.question_patterns = {
            'clarity': {
                'clear_starters': [r'^What\s+', r'^How\s+', r'^When\s+', r'^Where\s+', r'^Why\s+', r'^Which\s+', r'^Is\s+', r'^Are\s+', r'^Does\s+', r'^Did\s+', r'^Can\s+', r'^Could\s+', r'^Would\s+'],
                'vague_terms': [r'\bstuff\b', r'\bthing\b', r'\bsomething\b', r'\bit\b(?!\s+is)', r'\bthat\b(?!\s+is)'],
                'specific_terms': [r'\b\d+\b', r'\b(exactly|specifically|precisely)\b', r'\b(which|what type of|what kind of)\b']
            },
            'relevance': {
                'task_keywords': [r'\b(yes|no|true|false|correct|answer)\b', r'\b(problem|question|task)\b'],
                'context_references': [r'\b(given|provided|mentioned|stated)\b', r'\b(according to|based on)\b']
            }
        }
        
        # Reasoning faithfulness patterns
        self.reasoning_patterns = {
            'evidence_markers': [r'\b(because|since|due to|given that)\b', r'\b(evidence|proof|fact|data)\b'],
            'logical_connectors': [r'\b(therefore|thus|hence|consequently)\b', r'\b(if|then|when|while)\b'],
            'uncertainty_markers': [r'\b(might|could|possibly|likely|probably)\b', r'\b(seems|appears|suggests)\b'],
            'confidence_markers': [r'\b(definitely|certainly|clearly|obviously)\b', r'\b(must|will|always)\b']
        }
        
        print("✅ Enhanced Q&A-CoT evaluator initialized")
    
    def _extract_qa_pairs(self, response: str) -> List[Tuple[str, str]]:
        """Extract question-answer pairs from Q&A-CoT response."""
        qa_pairs = []
        
        # Pattern to match Question X: ... Answer X: ...
        qa_pattern = r'Question\s+(\d+):\s*([^\n]+?)\s*Answer\s+\1:\s*([^\n]+?)(?=Question\s+\d+:|Therefore|The answer is|$)'
        
        matches = re.findall(qa_pattern, response, re.IGNORECASE | re.DOTALL)
        
        for match in matches:
            question_num, question, answer = match
            qa_pairs.append((question.strip(), answer.strip()))
        
        return qa_pairs
    
    def _evaluate_question_quality(self, qa_pairs: List[Tuple[str, str]], original_question: str) -> QuestionQualityMetrics:
        """Evaluate the quality of generated questions."""
        if not qa_pairs:
            return QuestionQualityMetrics(0.0, 0.0, 0.0, 0.0, 0, 0.0)
        
        questions = [qa[0] for qa in qa_pairs]
        
        # Clarity score
        clarity_scores = []
        for q in questions:
            clear_count = sum(1 for pattern in self.question_patterns['clarity']['clear_starters'] if re.search(pattern, q, re.IGNORECASE))
            vague_count = sum(1 for pattern in self.question_patterns['clarity']['vague_terms'] if re.search(pattern, q, re.IGNORECASE))
            specific_count = sum(1 for pattern in self.question_patterns['clarity']['specific_terms'] if re.search(pattern, q, re.IGNORECASE))
            
            clarity = (clear_count + specific_count * 0.5) / max(1, len(q.split())) - vague_count * 0.2
            clarity_scores.append(max(0, min(1, clarity)))
        
        # Relevance score
        relevance_scores = []
        original_words = set(original_question.lower().split())
        
        for q in questions:
            q_words = set(q.lower().split())
            word_overlap = len(original_words.intersection(q_words)) / max(1, len(original_words))
            
            task_relevance = sum(1 for pattern in self.question_patterns['relevance']['task_keywords'] if re.search(pattern, q, re.IGNORECASE))
            context_relevance = sum(1 for pattern in self.question_patterns['relevance']['context_references'] if re.search(pattern, q, re.IGNORECASE))
            
            relevance = (word_overlap + task_relevance * 0.3 + context_relevance * 0.2) / 1.5
            relevance_scores.append(max(0, min(1, relevance)))
        
        # Progression score (how well questions build upon each other)
        progression_score = 0.0
        if len(questions) > 1:
            for i in range(1, len(questions)):
                prev_words = set(questions[i-1].lower().split())
                curr_words = set(questions[i].lower().split())
                overlap = len(prev_words.intersection(curr_words)) / max(1, len(prev_words.union(curr_words)))
                progression_score += overlap
            progression_score /= (len(questions) - 1)
        
        # Specificity score
        specificity_scores = []
        for q in questions:
            specific_count = sum(1 for pattern in self.question_patterns['clarity']['specific_terms'] if re.search(pattern, q, re.IGNORECASE))
            specificity = specific_count / max(1, len(q.split()))
            specificity_scores.append(min(1, specificity))
        
        return QuestionQualityMetrics(
            clarity_score=np.mean(clarity_scores),
            relevance_score=np.mean(relevance_scores),
            progression_score=progression_score,
            specificity_score=np.mean(specificity_scores),
            num_questions=len(questions),
            avg_question_length=np.mean([len(q.split()) for q in questions])
        )
    
    def _evaluate_reasoning_faithfulness(self, qa_pairs: List[Tuple[str, str]], final_answer: str) -> ReasoningFaithfulnessMetrics:
        """Evaluate the faithfulness of reasoning in answers."""
        if not qa_pairs:
            return ReasoningFaithfulnessMetrics(0.0, 0.0, 0.0, 0.0, 1.0)
        
        answers = [qa[1] for qa in qa_pairs]
        all_text = ' '.join(answers + [final_answer])
        
        # Evidence support score
        evidence_count = sum(1 for pattern in self.reasoning_patterns['evidence_markers'] if re.search(pattern, all_text, re.IGNORECASE))
        evidence_support = min(1.0, evidence_count / max(1, len(answers)))
        
        # Logical coherence score
        logical_count = sum(1 for pattern in self.reasoning_patterns['logical_connectors'] if re.search(pattern, all_text, re.IGNORECASE))
        logical_coherence = min(1.0, logical_count / max(1, len(answers)))
        
        # Answer alignment (how well intermediate answers lead to final answer)
        alignment_scores = []
        final_words = set(final_answer.lower().split())
        
        for answer in answers:
            answer_words = set(answer.lower().split())
            overlap = len(final_words.intersection(answer_words)) / max(1, len(final_words.union(answer_words)))
            alignment_scores.append(overlap)
        
        answer_alignment = np.mean(alignment_scores) if alignment_scores else 0.0
        
        # Factual consistency (simplified heuristic)
        confidence_markers = sum(1 for pattern in self.reasoning_patterns['confidence_markers'] if re.search(pattern, all_text, re.IGNORECASE))
        uncertainty_markers = sum(1 for pattern in self.reasoning_patterns['uncertainty_markers'] if re.search(pattern, all_text, re.IGNORECASE))
        
        # Higher confidence with lower uncertainty suggests better factual consistency
        factual_consistency = (confidence_markers - uncertainty_markers * 0.5) / max(1, len(answers))
        factual_consistency = max(0, min(1, factual_consistency + 0.5))  # Normalize to 0-1
        
        # Hallucination rate (simplified)
        # Look for contradictions or obviously false statements
        contradiction_patterns = [r'\b(not|never|impossible)\b.*\b(always|definitely|must)\b', 
                                r'\b(yes)\b.*\b(no)\b', r'\b(true)\b.*\b(false)\b']
        
        contradiction_count = sum(1 for pattern in contradiction_patterns if re.search(pattern, all_text, re.IGNORECASE))
        hallucination_rate = min(1.0, contradiction_count / max(1, len(answers)))
        
        return ReasoningFaithfulnessMetrics(
            factual_consistency=factual_consistency,
            logical_coherence=logical_coherence,
            evidence_support=evidence_support,
            answer_alignment=answer_alignment,
            hallucination_rate=hallucination_rate
        )
    
    def _extract_final_answer(self, response: str) -> str:
        """Extract the final answer from the response."""
        # Look for patterns like "The answer is **Yes**" or "Therefore, the answer is No"
        final_answer_patterns = [
            r'The answer is \*\*([^*]+)\*\*',
            r'Therefore[^.]*the answer is \*\*([^*]+)\*\*',
            r'The answer is ([YesNo]+)',
            r'Therefore[^.]*the answer is ([YesNo]+)'
        ]
        
        for pattern in final_answer_patterns:
            match = re.search(pattern, response, re.IGNORECASE)
            if match:
                return match.group(1).strip()
        
        # Fallback: look for Yes/No at the end
        if re.search(r'\b(Yes|No)\b(?!.*\b(Yes|No)\b)', response, re.IGNORECASE):
            match = re.search(r'\b(Yes|No)\b(?!.*\b(Yes|No)\b)', response, re.IGNORECASE)
            return match.group(1)
        
        return "Unknown"
    
    def evaluate_enhanced_response(self, question: str, response: str, ground_truth: str) -> Dict[str, Any]:
        """Evaluate a single Q&A-CoT response with enhanced metrics."""
        
        # Extract Q&A pairs and final answer
        qa_pairs = self._extract_qa_pairs(response)
        predicted_answer = self._extract_final_answer(response)
        
        # Basic accuracy
        is_correct = predicted_answer.lower() == ground_truth.lower()
        
        # Question quality metrics
        question_quality = self._evaluate_question_quality(qa_pairs, question)
        
        # Reasoning faithfulness metrics
        reasoning_faithfulness = self._evaluate_reasoning_faithfulness(qa_pairs, predicted_answer)
        
        # Format compliance
        has_qa_format = len(qa_pairs) > 0
        has_final_answer = predicted_answer != "Unknown"
        format_compliance = (has_qa_format + has_final_answer) / 2.0
        
        # Reasoning depth (based on number of Q&A pairs)
        reasoning_depth = min(1.0, len(qa_pairs) / 3.0)  # Normalize to 0-1, optimal around 3 questions
        
        return {
            'correct': is_correct,
            'predicted_answer': predicted_answer,
            'question_quality': question_quality,
            'reasoning_faithfulness': reasoning_faithfulness,
            'format_compliance': format_compliance,
            'reasoning_depth': reasoning_depth,
            'num_qa_pairs': len(qa_pairs)
        }
    
    def evaluate_dataset(self, model, questions: List[str], ground_truths: List[str], batch_size: int = 8) -> EnhancedEvaluationResults:
        """Evaluate entire dataset with enhanced metrics."""
        
        print(f"\n🔍 ENHANCED EVALUATION: Analyzing {len(questions)} examples...")
        print("Metrics: Accuracy + Question Quality + Reasoning Faithfulness")
        
        results = []
        correct_count = 0
        
        for i in range(0, len(questions), batch_size):
            batch_questions = questions[i:i+batch_size]
            batch_truths = ground_truths[i:i+batch_size]
            
            # Generate responses
            batch_responses = []
            for question in batch_questions:
                inputs = self.tokenizer(question, return_tensors="pt", truncation=True, max_length=512)
                inputs = {k: v.to(model.device) for k, v in inputs.items()}
                
                with torch.no_grad():
                    outputs = model.generate(
                        **inputs,
                        max_new_tokens=EVAL_MAX_TOKENS,
                        do_sample=True,
                        temperature=0.7,
                        pad_token_id=self.tokenizer.eos_token_id
                    )
                
                response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
                # Remove input from response
                response = response[len(question):].strip()
                batch_responses.append(response)
            
            # Evaluate batch
            for question, response, truth in zip(batch_questions, batch_responses, batch_truths):
                result = self.evaluate_enhanced_response(question, response, truth)
                results.append(result)
                if result['correct']:
                    correct_count += 1
            
            print(f"Processed {min(i+batch_size, len(questions))}/{len(questions)} examples...")
        
        # Aggregate metrics
        accuracy = correct_count / len(questions)
        
        # Question quality aggregation
        question_quality_agg = QuestionQualityMetrics(
            clarity_score=np.mean([r['question_quality'].clarity_score for r in results]),
            relevance_score=np.mean([r['question_quality'].relevance_score for r in results]),
            progression_score=np.mean([r['question_quality'].progression_score for r in results]),
            specificity_score=np.mean([r['question_quality'].specificity_score for r in results]),
            num_questions=np.mean([r['question_quality'].num_questions for r in results]),
            avg_question_length=np.mean([r['question_quality'].avg_question_length for r in results])
        )
        
        # Reasoning faithfulness aggregation
        reasoning_faithfulness_agg = ReasoningFaithfulnessMetrics(
            factual_consistency=np.mean([r['reasoning_faithfulness'].factual_consistency for r in results]),
            logical_coherence=np.mean([r['reasoning_faithfulness'].logical_coherence for r in results]),
            evidence_support=np.mean([r['reasoning_faithfulness'].evidence_support for r in results]),
            answer_alignment=np.mean([r['reasoning_faithfulness'].answer_alignment for r in results]),
            hallucination_rate=np.mean([r['reasoning_faithfulness'].hallucination_rate for r in results])
        )
        
        # Confidence calibration
        format_compliance = np.mean([r['format_compliance'] for r in results])
        reasoning_depth = np.mean([r['reasoning_depth'] for r in results])
        
        # Confidence calibration by correctness
        correct_results = [r for r in results if r['correct']]
        incorrect_results = [r for r in results if not r['correct']]
        
        confidence_calibration = {
            'overall_format_compliance': format_compliance,
            'correct_format_compliance': np.mean([r['format_compliance'] for r in correct_results]) if correct_results else 0.0,
            'incorrect_format_compliance': np.mean([r['format_compliance'] for r in incorrect_results]) if incorrect_results else 0.0,
            'reasoning_depth_difference': (
                np.mean([r['reasoning_depth'] for r in correct_results]) - 
                np.mean([r['reasoning_depth'] for r in incorrect_results])
            ) if correct_results and incorrect_results else 0.0
        }
        
        return EnhancedEvaluationResults(
            accuracy=accuracy,
            question_quality=question_quality_agg,
            reasoning_faithfulness=reasoning_faithfulness_agg,
            confidence_calibration=confidence_calibration,
            format_compliance=format_compliance,
            reasoning_depth=reasoning_depth,
            total_examples=len(questions)
        )

# ============================================================================
# EXECUTE ENHANCED EVALUATION
# ============================================================================

print("\n🔍 LAUNCHING ENHANCED EVALUATION WITH QUESTION QUALITY & REASONING FAITHFULNESS")
print("==================================================================================")

# Initialize enhanced evaluator
enhanced_evaluator = EnhancedQACoTEvaluator(tokenizer)

# Load evaluation dataset (use a subset for faster evaluation)
eval_size = 100  # Adjust based on computational resources
eval_questions = []
eval_answers = []

# Load from validation data if available
if os.path.exists(val_file):
    print(f"Loading evaluation data from: {val_file}")
    with open(val_file, 'r') as f:
        eval_data = [json.loads(line) for line in f.readlines()[:eval_size]]
    
    for item in eval_data:
        eval_questions.append(item['prompt'])
        eval_answers.append(item['answer'])
        
    print(f"📊 Loaded {len(eval_questions)} evaluation examples")
    
    # Run enhanced evaluation
    print(f"\n🚀 Starting enhanced evaluation on {len(eval_questions)} examples...")
    
    enhanced_results = enhanced_evaluator.evaluate_dataset(
        model=model,
        questions=eval_questions,
        ground_truths=eval_answers,
        batch_size=ENHANCED_EVAL_BATCH_SIZE  # Smaller batch size for detailed analysis
    )
    
    # Display enhanced evaluation results
    print("\n=== ENHANCED EVALUATION RESULTS ===")
    print(f"📊 Overall Accuracy: {enhanced_results.accuracy:.1%}")
    print(f"📝 Format Compliance: {enhanced_results.format_compliance:.1%}")
    print(f"🧠 Reasoning Depth: {enhanced_results.reasoning_depth:.3f}")
    
    print("\n🔍 QUESTION QUALITY METRICS:")
    qm = enhanced_results.question_quality
    print(f"  • Clarity Score: {qm.clarity_score:.3f}")
    print(f"  • Relevance Score: {qm.relevance_score:.3f}")
    print(f"  • Progression Score: {qm.progression_score:.3f}")
    print(f"  • Specificity Score: {qm.specificity_score:.3f}")
    print(f"  • Avg Questions per Response: {qm.num_questions:.1f}")
    print(f"  • Avg Question Length: {qm.avg_question_length:.1f} words")
    
    print("\n🧠 REASONING FAITHFULNESS METRICS:")
    rm = enhanced_results.reasoning_faithfulness
    print(f"  • Factual Consistency: {rm.factual_consistency:.3f}")
    print(f"  • Logical Coherence: {rm.logical_coherence:.3f}")
    print(f"  • Evidence Support: {rm.evidence_support:.3f}")
    print(f"  • Answer Alignment: {rm.answer_alignment:.3f}")
    print(f"  • Hallucination Rate: {rm.hallucination_rate:.3f} (lower is better)")
    
    print("\n📈 CONFIDENCE CALIBRATION:")
    cc = enhanced_results.confidence_calibration
    print(f"  • Correct Answer Format Compliance: {cc['correct_format_compliance']:.1%}")
    print(f"  • Incorrect Answer Format Compliance: {cc['incorrect_format_compliance']:.1%}")
    print(f"  • Reasoning Depth Difference (Correct - Incorrect): {cc['reasoning_depth_difference']:.3f}")
    
    # Calculate overall quality score
    question_quality_score = (qm.clarity_score + qm.relevance_score + qm.progression_score + qm.specificity_score) / 4
    reasoning_quality_score = (rm.factual_consistency + rm.logical_coherence + rm.evidence_support + rm.answer_alignment - rm.hallucination_rate) / 4
    overall_quality_score = (enhanced_results.accuracy + question_quality_score + reasoning_quality_score + enhanced_results.format_compliance) / 4
    
    print(f"\n🎯 OVERALL QUALITY SCORES:")
    print(f"  • Question Quality Score: {question_quality_score:.3f}")
    print(f"  • Reasoning Quality Score: {reasoning_quality_score:.3f}")
    print(f"  • Overall Model Quality: {overall_quality_score:.3f}")
    
    # Save enhanced evaluation results
    enhanced_eval_results = {
        'accuracy': enhanced_results.accuracy,
        'format_compliance': enhanced_results.format_compliance,
        'reasoning_depth': enhanced_results.reasoning_depth,
        'question_quality': {
            'clarity_score': qm.clarity_score,
            'relevance_score': qm.relevance_score,
            'progression_score': qm.progression_score,
            'specificity_score': qm.specificity_score,
            'num_questions': qm.num_questions,
            'avg_question_length': qm.avg_question_length
        },
        'reasoning_faithfulness': {
            'factual_consistency': rm.factual_consistency,
            'logical_coherence': rm.logical_coherence,
            'evidence_support': rm.evidence_support,
            'answer_alignment': rm.answer_alignment,
            'hallucination_rate': rm.hallucination_rate
        },
        'confidence_calibration': cc,
        'quality_scores': {
            'question_quality_score': question_quality_score,
            'reasoning_quality_score': reasoning_quality_score,
            'overall_model_quality': overall_quality_score
        },
        'total_examples': enhanced_results.total_examples
    }
    
    enhanced_eval_file = os.path.join(parent_dir, 'enhanced_evaluation_results.json')
    with open(enhanced_eval_file, 'w') as f:
        json.dump(enhanced_eval_results, f, indent=2)
    
    print(f"\n📊 Enhanced evaluation results saved to: {enhanced_eval_file}")
    
    print("\n🎯 Enhanced evaluation completed!")
    print("✅ Question quality assessment")
    print("✅ Reasoning faithfulness analysis")
    print("✅ Confidence calibration metrics")
    print("✅ Comprehensive quality scoring")
    
else:
    print(f"⚠️ Validation file not found: {val_file}")
    print("Please ensure validation data is available for enhanced evaluation.")

# Self‑Improving LLM Project

This notebook implements Parts 2 and 3 of the project plan for the **Self‑Improving LLM** final project.  Specifically, it covers:

- **Dataset Acquisition & Sampling:** download the StrategyQA dataset, sample ~2 000 training examples as recommended, and save them to disk for subsequent processing.
- **Prompt Engineering & Teacher Generation:** generate a baseline *student draft* for each question, compose prompts according to the plan (question, student draft, and a teacher instruction), call GPT‑4 (or run in dry‑run mode), and build two parallel corpora for baseline and CoT training.

The plan specifies a data‑generation loop where each question is paired with a student draft and a teacher chain‑of‑thought, resulting in two training tracks.  The baseline model is trained on `(Q → answer)` pairs, while the CoT model is trained on `(Q + teacher CoT → answer)` pair.

> **Note:** Running the full pipeline (especially calling GPT‑4) requires an OpenAI API key and may incur costs.  A dry‑run mode is provided for testing the notebook without external API calls.


In [1]:
!pip install -q datasets transformers openai bitsandbytes accelerate python-dotenv huggingface_hub huggingface_hub[hf_xet]


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file if it exists
load_dotenv()

# Dataset parameters
DATASET_NAME = os.getenv('DATASET_NAME', 'voidful/StrategyQA')
TRAIN_SAMPLES = int(os.getenv('TRAIN_SAMPLES', '100'))
RANDOM_SEED = int(os.getenv('RANDOM_SEED', '42'))
USE_FULL_DATASET = os.getenv('USE_FULL_DATASET', 'False').lower() in ('true', '1', 't')


# Model parameters
MODEL_NAME = os.getenv('MODEL_NAME', 'microsoft/phi-2')
MAX_NEW_TOKENS = int(os.getenv('MAX_NEW_TOKENS', '35'))
BATCH_SIZE = int(os.getenv('BATCH_SIZE', '8'))
USE_4BIT = os.getenv('USE_4BIT', 'True').lower() in ('true', '1', 't')
MAX_SEQ_LENGTH = int(os.getenv('MAX_SEQ_LENGTH', '512'))
HUGGINGFACE_TOKEN = os.getenv('HUGGINGFACE_TOKEN', '')

# Generation parameters
DO_SAMPLE = os.getenv('DO_SAMPLE', 'False').lower() in ('true', '1', 't')
TEMPERATURE = float(os.getenv('TEMPERATURE', '0.7'))

# GPT-4 parameters
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY', '')
GPT4_MODEL = os.getenv('GPT4_MODEL', 'gpt-4')
GPT4_MAX_TOKENS = int(os.getenv('GPT4_MAX_TOKENS', '150'))
GPT4_TEMPERATURE = float(os.getenv('GPT4_TEMPERATURE', '0.3'))
DRY_RUN = os.getenv('DRY_RUN', 'True').lower() in ('true', '1', 't')

# Student Draft Generation
STUDENT_MAX_TOKENS = int(os.getenv('STUDENT_MAX_TOKENS', '200'))
STUDENT_TEMPERATURE = float(os.getenv('STUDENT_TEMPERATURE', '0.7'))
STUDENT_BATCH_SIZE = int(os.getenv('STUDENT_BATCH_SIZE', '8'))

# Enhanced Evaluation Generation
EVAL_MAX_TOKENS = int(os.getenv('EVAL_MAX_TOKENS', '256'))
EVAL_TEMPERATURE = float(os.getenv('EVAL_TEMPERATURE', '0.7'))
EVAL_BATCH_SIZE = int(os.getenv('EVAL_BATCH_SIZE', '4'))

# Quick Evaluation
QUICK_EVAL_MAX_TOKENS = int(os.getenv('QUICK_EVAL_MAX_TOKENS', '5'))

# Training Configuration
# Phase A Training
PHASE_A_EPOCHS = int(os.getenv('PHASE_A_EPOCHS', '3'))
PHASE_A_BATCH_SIZE = int(os.getenv('PHASE_A_BATCH_SIZE', '1'))
PHASE_A_LEARNING_RATE = float(os.getenv('PHASE_A_LEARNING_RATE', '1e-4'))
PHASE_A_WARMUP_RATIO = float(os.getenv('PHASE_A_WARMUP_RATIO', '0.1'))
PHASE_A_WEIGHT_DECAY = float(os.getenv('PHASE_A_WEIGHT_DECAY', '0.01'))

# Phase B Training
PHASE_B_EPOCHS = int(os.getenv('PHASE_B_EPOCHS', '3'))
PHASE_B_BATCH_SIZE = int(os.getenv('PHASE_B_BATCH_SIZE', '4'))
PHASE_B_LEARNING_RATE = float(os.getenv('PHASE_B_LEARNING_RATE', '5e-5'))
PHASE_B_WARMUP_RATIO = float(os.getenv('PHASE_B_WARMUP_RATIO', '0.1'))
PHASE_B_WEIGHT_DECAY = float(os.getenv('PHASE_B_WEIGHT_DECAY', '0.01'))

# Progressive Curriculum Training
CURRICULUM_STAGE1_EPOCHS = int(os.getenv('CURRICULUM_STAGE1_EPOCHS', '1'))
CURRICULUM_STAGE1_LEARNING_RATE = float(os.getenv('CURRICULUM_STAGE1_LEARNING_RATE', '5e-5'))
CURRICULUM_STAGE1_WARMUP_RATIO = float(os.getenv('CURRICULUM_STAGE1_WARMUP_RATIO', '0.1'))
CURRICULUM_STAGE1_WEIGHT_DECAY = float(os.getenv('CURRICULUM_STAGE1_WEIGHT_DECAY', '0.01'))

CURRICULUM_STAGE2_EPOCHS = int(os.getenv('CURRICULUM_STAGE2_EPOCHS', '2'))
CURRICULUM_STAGE2_LEARNING_RATE = float(os.getenv('CURRICULUM_STAGE2_LEARNING_RATE', '3e-5'))
CURRICULUM_STAGE2_WARMUP_RATIO = float(os.getenv('CURRICULUM_STAGE2_WARMUP_RATIO', '0.1'))
CURRICULUM_STAGE2_WEIGHT_DECAY = float(os.getenv('CURRICULUM_STAGE2_WEIGHT_DECAY', '0.01'))

# Validation Configuration
HIGH_CONFIDENCE_THRESHOLD = float(os.getenv('HIGH_CONFIDENCE_THRESHOLD', '0.8'))
MEDIUM_CONFIDENCE_THRESHOLD = float(os.getenv('MEDIUM_CONFIDENCE_THRESHOLD', '0.5'))
LOW_CONFIDENCE_THRESHOLD = float(os.getenv('LOW_CONFIDENCE_THRESHOLD', '0.3'))
VALIDATION_ACCEPTANCE_THRESHOLD = float(os.getenv('VALIDATION_ACCEPTANCE_THRESHOLD', '0.3'))

# Quality Thresholds
VALID_QUALITY_THRESHOLD = float(os.getenv('VALID_QUALITY_THRESHOLD', '0.5'))
CORRECTED_QUALITY_THRESHOLD = float(os.getenv('CORRECTED_QUALITY_THRESHOLD', '0.3'))

# Token Emphasis Configuration
EMPHASIS_MULTIPLIER = float(os.getenv('EMPHASIS_MULTIPLIER', '2.5'))
ADAPTIVE_EMPHASIS = os.getenv('ADAPTIVE_EMPHASIS', 'True').lower() in ('true', '1', 't')

# Memory and Performance
GRADIENT_CHECKPOINTING = os.getenv('GRADIENT_CHECKPOINTING', 'True').lower() in ('true', '1', 't')
FP16 = os.getenv('FP16', 'False').lower() in ('true', '1', 't')
BF16 = os.getenv('BF16', 'True').lower() in ('true', '1', 't')
DATALOADER_NUM_WORKERS = int(os.getenv('DATALOADER_NUM_WORKERS', '0'))
DATALOADER_PERSISTENT_WORKERS = os.getenv('DATALOADER_PERSISTENT_WORKERS', 'False').lower() in ('true', '1', 't')
SKIP_MEMORY_METRICS = os.getenv('SKIP_MEMORY_METRICS', 'True').lower() in ('true', '1', 't')

# Logging and Monitoring
LOGGING_STEPS = int(os.getenv('LOGGING_STEPS', '10'))
SAVE_STRATEGY = os.getenv('SAVE_STRATEGY', 'epoch')
REPORT_TO = os.getenv('REPORT_TO', 'none')
LOAD_BEST_MODEL_AT_END = os.getenv('LOAD_BEST_MODEL_AT_END', 'False').lower() in ('true', '1', 't')

# Evaluation Configuration
EVAL_SIZE = int(os.getenv('EVAL_SIZE', '100'))
ENHANCED_EVAL_BATCH_SIZE = int(os.getenv('ENHANCED_EVAL_BATCH_SIZE', '4'))
# File paths
parent_dir = os.path.dirname(os.getcwd())
DATA_DIR = os.path.join(parent_dir, os.getenv("DATA_DIR", "data"))
RAW_DIR = os.path.join(DATA_DIR, 'raw')
TRAIN_DIR = os.path.join(DATA_DIR, 'train')
STUDENT_DIR = os.path.join(DATA_DIR, 'student')
TEACHER_DIR = os.path.join(DATA_DIR, 'teacher')
SAMPLE_TRAIN_PATH = os.path.join(TRAIN_DIR, 'sample_train.jsonl')
STUDENT_DRAFTS_PATH = os.path.join(STUDENT_DIR, 'student_drafts.jsonl')
CLEANED_STUDENT_DRAFTS_PATH = os.path.join(STUDENT_DIR, 'cleaned_student_drafts.jsonl')
TEACHER_OUTPUTS_PATH = os.path.join(TEACHER_DIR, 'teacher_outputs.jsonl')
BASELINE_PATH = os.path.join(TRAIN_DIR, 'train_baseline.jsonl')
COT_PATH = os.path.join(TRAIN_DIR, 'train_cot.jsonl')
COT_PATH_QA_COT = os.path.join(TEACHER_DIR, 'teacher_outputs_qa_cot.jsonl')
SAMPLE_TEST_PATH = SAMPLE_TRAIN_PATH  # Alias for consistency with Build Training Corpora cell

# Print configuration
print("=== Configuration ===")
print(f"Dataset: {DATASET_NAME}")
print(f"Model: {MODEL_NAME}")
print(f"Batch size: {BATCH_SIZE}")
print(f"4-bit quantization: {USE_4BIT}")
print(f"GPT-4 dry run: {DRY_RUN}")
print("=="*10)

=== Configuration ===
Dataset: voidful/StrategyQA
Model: microsoft/Phi-3.5-mini-instruct
Batch size: 8
4-bit quantization: True
GPT-4 dry run: False


In [ ]:
# from huggingface_hub import login, notebook_login

# def smart_hf_login():
#     """Use HF_TOKEN env/secret if present, else fall back to interactive login."""
#     if HUGGINGFACE_TOKEN:         # works for Colab secrets, CI, docker, …
#         login(HUGGINGFACE_TOKEN)
#     elif 'google.colab' in sys.modules:   # inside a Colab kernel but no secret set
#         notebook_login()
#     else:                                 # local Jupyter; will prompt only once
#         login()

# smart_hf_login()


In [ ]:
from datasets import load_dataset
import json
import os

def save_jsonl(data, filepath):
    """Save data to a JSONL file."""
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    with open(filepath, 'w', encoding='utf-8') as f:
        for item in data:
            f.write(json.dumps(item) + '\n')

def load_jsonl(filepath):
    """Load data from a JSONL file."""
    with open(filepath, 'r', encoding='utf-8') as f:
        return [json.loads(line) for line in f]

# Check if files exist
train_path = os.path.join(RAW_DIR, 'strategyqa_train.jsonl')
val_path = os.path.join(RAW_DIR, 'strategyqa_validation.jsonl')
test_path = os.path.join(RAW_DIR, 'strategyqa_test.jsonl')

print("Looking for files in:")
print(f"- Train: {train_path}")
print(f"- Val: {val_path}")
print(f"- Test: {test_path}")

# Create data directory
os.makedirs(RAW_DIR, exist_ok=True)

files_exist = all(os.path.exists(p) for p in [train_path, val_path, test_path])
print(f"Files exist: {files_exist}")

if files_exist:
    print("Loading dataset from local files...")
    train_data = load_jsonl(train_path)
    val_data = load_jsonl(val_path)
    test_data = load_jsonl(test_path)
else:
    print("Downloading and saving dataset...")
    # Load the dataset from HuggingFace
    dataset = load_dataset(DATASET_NAME)

    train_data = list(dataset['train'])
    val_data = list(dataset['validation'])
    test_data = list(dataset['test'])

    # Save to files for future use
    save_jsonl(train_data, train_path)
    save_jsonl(val_data, val_path)
    save_jsonl(test_data, test_path)

    print(f"Train: {len(train_data)} examples")
    print(f"Val: {len(val_data)} examples")
    print(f"Test: {len(test_data)} examples")

# Create sample training data
sample_train_path = os.path.join(TRAIN_DIR, 'sample_train.jsonl')

if not USE_FULL_DATASET:
    # Create a smaller sample for faster development
    import random
    random.seed(RANDOM_SEED)
    target_train_sampled = random.sample(train_data, min(TRAIN_SAMPLES, len(train_data)))
    print(f"Sample size: {len(target_train_sampled)} examples")
else:
    # Use all training data
    target_train_sampled = train_data
    print(f"Using full training set: {len(target_train_sampled)} examples")

# Also create combined train+validation for Q&A-CoT (more data)
full_train_val = train_data + val_data
full_train_val_path = os.path.join(TRAIN_DIR, 'full_train_val.jsonl')

# Save both sampled and full datasets
save_jsonl(target_train_sampled, sample_train_path)
save_jsonl(full_train_val, full_train_val_path)

print(f"Full training set saved to {train_path}")
print(f"Validation set saved to {val_path}")
print(f"Sampled train set (≈{TRAIN_SAMPLES} entries) saved to {sample_train_path}")
print(f"Combined train+val set ({len(full_train_val)} entries) saved to {full_train_val_path}")

# Update file paths based on choice
if USE_FULL_DATASET:
    # Update the global path variables to point to the full dataset
    SAMPLE_TRAIN_PATH = os.path.join(DATA_DIR, 'full_train_val.jsonl')
    print(f"📊 Updated SAMPLE_TRAIN_PATH to use full dataset: {SAMPLE_TRAIN_PATH}")
    # Update file names to avoid confusion
    STUDENT_DRAFTS_PATH = os.path.join(DATA_DIR, 'student_drafts_full.jsonl')
    CLEANED_STUDENT_DRAFTS_PATH = os.path.join(DATA_DIR, 'cleaned_student_drafts_full.jsonl')
    TEACHER_OUTPUTS_PATH = os.path.join(DATA_DIR, 'teacher_outputs_full.jsonl')
    BASELINE_PATH = os.path.join(DATA_DIR, 'train_baseline_full.jsonl')
    COT_PATH = os.path.join(DATA_DIR, 'train_cot_full.jsonl')
    COT_PATH_QA_COT = os.path.join(TEACHER_DIR, 'teacher_outputs_full.jsonl')
    SAMPLE_TEST_PATH = SAMPLE_TRAIN_PATH  # Updated alias
    print(f"📊 Updated output paths to use '_full' suffix for clarity")

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from torch.utils.data import DataLoader
from datasets import load_dataset
import torch
import json
import os
from tqdm import tqdm

def setup_dataset(input_path: str, tokenizer, batch_size: int = BATCH_SIZE):
    """Load and prepare dataset for GPU processing."""
    # Load the dataset
    dataset = load_dataset('json', data_files=input_path, split='train')

    # Keep the original questions for reference
    original_questions = dataset['question']

    # Tokenization function
    def tokenize_function(examples):
        return tokenizer(
            examples['question'],
            truncation=True,
            padding='max_length',
            max_length=MAX_SEQ_LENGTH,
            return_tensors=None  # Return as list, not tensors
        )

    # Apply tokenization
    tokenized_dataset = dataset.map(tokenize_function, batched=True)

    # Create a custom dataset that includes both tokenized data and original questions
    class QADataset(torch.utils.data.Dataset):
        def __init__(self, tokenized_data, original_questions):
            self.tokenized_data = tokenized_data
            self.original_questions = original_questions

        def __len__(self):
            return len(self.tokenized_data)

        def __getitem__(self, idx):
            item = {
                'input_ids': torch.tensor(self.tokenized_data[idx]['input_ids']),
                'attention_mask': torch.tensor(self.tokenized_data[idx]['attention_mask']),
                'question': self.original_questions[idx]
            }
            return item

    # Create custom dataset
    custom_dataset = QADataset(tokenized_dataset, original_questions)

    # Create DataLoader
    loader = DataLoader(
        custom_dataset,
        batch_size=batch_size,
        shuffle=False  # Keep order for output matching
    )

    return loader

# GPU setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Model setup
print(f"Loading model: {MODEL_NAME}")

# Use 4-bit quantization if enabled and on GPU
if device.type == 'cuda' and USE_4BIT:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
    )
    print("Loading model in 4-bit quantization...")
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        quantization_config=bnb_config,
        device_map="auto"
    )
else:
    print("Loading model in standard precision...")
    model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(device)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# Ensure the tokenizer has a padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
model.eval()

if device.type == 'cuda':
    print(f"GPU Memory after model load: {torch.cuda.memory_allocated()/1e9:.2f} GB")

# Load and prepare dataset (fix path)
parent_dir = os.path.dirname(os.getcwd())
sample_train_path_full = os.path.join(parent_dir, SAMPLE_TRAIN_PATH)
print(f"Loading dataset from {sample_train_path_full} with batch size {BATCH_SIZE}")
train_loader = setup_dataset(sample_train_path_full, tokenizer, batch_size=BATCH_SIZE)

Using device: cuda
Loading model: microsoft/Phi-3.5-mini-instruct
Loading model in 4-bit quantization...


2025-08-19 16:14:31,097 - INFO - We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).
Loading checkpoint shards: 100%|██████████| 2/2 [00:14<00:00,  7.13s/it]


GPU Memory after model load: 2.26 GB
Loading dataset from c:\Users\noham\Desktop\Self-Improving-LLM\data\train\sample_train.jsonl with batch size 8


## Generate Student Drafts

In this section we load a base language model (e.g. `meta-llama/Llama-2-7b-hf` or `gpt2`) and generate a short *student draft* for each question in the sampled training set.  A draft consists of a yes/no answer followed by one or two clarifying questions, as specified in the data‑generation loop.  Adjust the model name based on your available hardware and licences.

> **Tip:** On Colab, you can enable a GPU via *Runtime → Change runtime type → GPU* and use half‑precision weights to reduce memory usage.  For demonstration, we use `gpt2` (which is small) to keep the example runnable on CPU.


## Generate Teacher Responses

We now call GPT‑4 to obtain chain‑of‑thought (CoT) reasoning and final yes/no answers for each question/draft pair.  The prompt format follows the plan:

```
Q: <original yes/no question>
Student draft: <answer + clarifying questions>
Teacher: Please think step-by-step and provide your thought process and final Yes/No answer.
```

To run the actual API calls, you must provide a valid OpenAI API key.  If you set `dry_run=True`, dummy responses will be generated for testing purposes.


## Build Training Corpora

Finally, we build two parallel training corpora:

1. **Baseline (Track A)** – pairs of `(question → answer)` for training a basic model.
2. **CoT (Track B)** – pairs of `(question + teacher chain-of-thought → answer)` for CoT distillation.

These files will be used in later steps for model fine‑tuning.


In [6]:
import json
import random
from collections import Counter
from typing import Dict, List, Any, Optional, Tuple
import logging
from dataclasses import dataclass
from enum import Enum
import re

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class ValidationStatus(Enum):
    VALID = "valid"
    CORRECTED = "corrected" 
    INVALID = "invalid"

@dataclass
class ValidationResult:
    status: ValidationStatus
    original_text: str
    cleaned_text: Optional[str]
    confidence_score: float  # 0.0 - 1.0
    error_messages: List[str]
    metadata: Dict[str, Any]

    def is_valid(self) -> bool:
        return self.status in [ValidationStatus.VALID, ValidationStatus.CORRECTED]


class ResponseValidationPipeline:
    """Professional validation pipeline with multi-layered validation and confidence scoring."""
    
    def __init__(self):
        """Initialize pipeline with professional standards."""
        self.teacher_validator = TeacherResponseValidator()
        self.HIGH_CONFIDENCE_THRESHOLD = 0.8
        self.MEDIUM_CONFIDENCE_THRESHOLD = 0.5

    def process_responses(self, teacher_data: List[Dict], ground_truth_map: Dict[str, bool]) -> Tuple[List[Dict], Dict[str, Any]]:
        """Process teacher responses with multi-layered validation and confidence scoring."""

        # Initialize comprehensive statistics
        validation_stats = {
            'total_processed': 0,
            'ground_truth_missing': 0,
            'data_kept': 0,

            # Validation tiers
            'high_confidence': 0,    # >= 0.8
            'medium_confidence': 0,  # 0.5 - 0.8
            'low_confidence': 0,     # < 0.5

            # Answer validation
            'valid_teacher_answers': 0,
            'invalid_teacher_answers': 0,
            'teacher_agrees': 0,
            'teacher_disagrees': 0,

            # Q&A-CoT format tracking
            'qa_cot_format': 0,
            'traditional_format': 0,

            # Error tracking 
            'validation_errors': Counter(),
            'correction_attempts': 0,
            'successful_corrections': 0,
            'confidence_scores': [],
            
            # Template fallback tracking
            'template_fallbacks': 0,  # Track samples that used template fallback
        }

        baseline_records = []
        cot_records = []
        HIGH_CONFIDENCE_THRESHOLD = self.HIGH_CONFIDENCE_THRESHOLD

        for rec in teacher_data:
            q = rec['question']
            thought = rec['teacher_thought']
            teacher_answer = rec['teacher_answer']
            format_type = rec.get('format_type', 'unknown')

            validation_stats['total_processed'] += 1

            # Track format types
            if format_type == 'qa_interleaved':
                validation_stats['qa_cot_format'] += 1
            else:
                validation_stats['traditional_format'] += 1

            # Check ground truth availability
            if q not in ground_truth_map:
                validation_stats['ground_truth_missing'] += 1
                continue

            ground_truth_answer = self._convert_to_yes_no(ground_truth_map[q])

            # Use existing validation metadata if available
            if 'validation_metadata' in rec:
                validation_result = self._create_validation_result_from_metadata(rec['validation_metadata'], thought)
            else:
                # Apply professional validation
                validation_result = self.teacher_validator.validate(thought)

            validation_stats['confidence_scores'].append(validation_result.confidence_score)

            # Track validation status
            if validation_result.status == ValidationStatus.VALID:
                validation_stats['high_confidence'] += 1 if validation_result.confidence_score >= HIGH_CONFIDENCE_THRESHOLD else 0
                validation_stats['medium_confidence'] += 1 if 0.5 <= validation_result.confidence_score < 0.8 else 0
            elif validation_result.status == ValidationStatus.CORRECTED:
                validation_stats['successful_corrections'] += 1
                validation_stats['medium_confidence'] += 1
            else:
                validation_stats['low_confidence'] += 1

            # Track errors
            for error in validation_result.error_messages:
                validation_stats['validation_errors'][error] += 1

            # Validate teacher answer format
            if not self._is_valid_answer(teacher_answer):
                validation_stats['invalid_teacher_answers'] += 1
                continue
            else:
                validation_stats['valid_teacher_answers'] += 1

            # Track teacher-ground truth agreement
            teacher_answer_clean = teacher_answer.strip().capitalize()
            if teacher_answer_clean == ground_truth_answer:
                validation_stats['teacher_agrees'] += 1
            else:
                validation_stats['teacher_disagrees'] += 1

            # Confidence-based processing decision
            confidence_tier = self._get_confidence_tier(validation_result.confidence_score)

            if confidence_tier in ['high', 'medium']:
                # Accept high and medium confidence responses
                validation_stats['data_kept'] += 1

                # Use validated response if available
                final_thought = validation_result.cleaned_text if validation_result.cleaned_text else thought

                # Create baseline record (question → teacher_answer)
                baseline_record = {
                    'prompt': q,
                    'answer': teacher_answer_clean,
                    'validation_metadata': {
                        'confidence_score': validation_result.confidence_score,
                        'validation_status': validation_result.status.value,
                        'confidence_tier': confidence_tier,
                        'format_type': format_type
                    }
                }
                baseline_records.append(baseline_record)

                # Create enhanced Q&A-CoT record
                if format_type == 'qa_interleaved':
                    # Use Q&A-CoT format for CoT training
                    cot_prompt, used_template = self._create_qa_cot_training_prompt(q, final_thought)
                    
                    # Track template usage
                    if used_template:
                        validation_stats['template_fallbacks'] = validation_stats.get('template_fallbacks', 0) + 1
                else:
                    # Use enhanced traditional format
                    cot_prompt = self._create_enhanced_cot_prompt(q, draft, final_thought)

                cot_record = {
                    'prompt': cot_prompt,
                    'answer': teacher_answer_clean,
                    'validation_metadata': {
                        'confidence_score': validation_result.confidence_score,
                        'validation_status': validation_result.status.value,
                        'confidence_tier': confidence_tier,
                        'format_type': format_type
                    }
                }
                cot_records.append(cot_record)

            # Progress reporting
            if validation_stats['total_processed'] % 100 == 0:
                print(f"Processed {validation_stats['total_processed']} responses...")

        # Calculate final statistics
        self._calculate_final_stats(validation_stats)

        return baseline_records, cot_records, validation_stats

    def _create_validation_result_from_metadata(self, metadata: Dict[str, Any], original_text: str) -> 'ValidationResult':
        """Create ValidationResult from existing metadata."""
        return ValidationResult(
            status=ValidationStatus(metadata.get('status', 'valid')),
            original_text=original_text,
            cleaned_text=original_text,  # Already processed
            confidence_score=metadata.get('confidence_score', 0.5),
            error_messages=metadata.get('errors', []),
            metadata=metadata
        )

    def _create_qa_cot_training_prompt(self, question: str, teacher_thought: str) -> tuple:
        """Create Q&A-CoT training prompt using actual teacher reasoning.
        
        Returns:
            tuple: (prompt_text, used_template_flag)
        """

        # Extract the Q&A structure from teacher thought
        qa_match = re.search(r'(Question\s+\d+:.*?Answer\s+\d+:.*?)(?=Therefore|The answer is|$)', teacher_thought, re.DOTALL | re.IGNORECASE)

        if qa_match:
            qa_content = qa_match.group(1).strip()
            
            # Extract the conclusion part (Therefore...) from teacher_thought
            conclusion_match = re.search(r'(Therefore.*?)(?=The answer is|$)', teacher_thought, re.DOTALL | re.IGNORECASE)
            conclusion = conclusion_match.group(1).strip() if conclusion_match else "Therefore, based on the analysis above"
            
            # Create training prompt with actual teacher reasoning
            prompt = f"""Question: {question}

                    {qa_content}
                    {conclusion}"""
            
            return prompt, False  # False = did not use template
            
        else:
            # Fallback template if Q&A structure not found
            prompt = f"""Question: {question}

                        Question 1: [Ask a clarifying question about this topic]
                        Answer 1: [Provide factual information]
                        Therefore, [conclude based on the analysis]"""
            
            return prompt, True  # True = used template fallback

    def _get_confidence_tier(self, score: float) -> str:
        """Classify confidence score into tier."""
        if score >= self.HIGH_CONFIDENCE_THRESHOLD:
            return 'high'
        elif score >= self.MEDIUM_CONFIDENCE_THRESHOLD:
            return 'medium'
        else:
            return 'low'

    def _create_enhanced_cot_prompt(self, question: str, draft: str, teacher_thought: str) -> str:
        """Create enhanced CoT prompt for traditional format."""
        return f"""Question: {question}

                    Draft: {draft}

                    Reasoning: {teacher_thought}"""

    def _is_valid_answer(self, answer: str) -> bool:
        """Check if answer is valid yes/no format."""
        if not answer or not isinstance(answer, str):
            return False
        cleaned = answer.strip().lower()
        return cleaned in ['yes', 'no']

    def _convert_to_yes_no(self, ground_truth: bool) -> str:
        """Convert boolean ground truth to Yes/No string."""
        return "Yes" if ground_truth else "No"

    def _calculate_final_stats(self, validation_stats: Dict[str, Any]) -> None:
        """Calculate final statistics."""
        total_processed = validation_stats['total_processed']
        
        if total_processed > 0:
            # Calculate agreement rate
            total_agreements = validation_stats['teacher_agrees'] + validation_stats['teacher_disagrees']
            if total_agreements > 0:
                validation_stats['teacher_agreement_rate'] = (validation_stats['teacher_agrees'] / total_agreements) * 100

            # Calculate average confidence
            if validation_stats['confidence_scores']:
                validation_stats['avg_confidence'] = sum(validation_stats['confidence_scores']) / len(validation_stats['confidence_scores'])
            else:
                validation_stats['avg_confidence'] = 0.0


class TeacherResponseValidator:
    """Validates teacher responses for quality and consistency."""
    
    def __init__(self):
        # Quality thresholds
        self.MIN_LENGTH = 50
        self.MAX_LENGTH = 2000
        self.REQUIRED_PATTERNS = [
            r'question\s+\d+:', r'answer\s+\d+:', r'therefore'
        ]
    
    def validate(self, text: str) -> ValidationResult:
        """Comprehensive validation of teacher response."""
        errors = []
        confidence_score = 1.0
        
        # Length check
        if len(text) < self.MIN_LENGTH:
            errors.append("Response too short")
            confidence_score *= 0.7
        elif len(text) > self.MAX_LENGTH:
            errors.append("Response too long")
            confidence_score *= 0.9
            
        # Pattern checks
        patterns_found = 0
        for pattern in self.REQUIRED_PATTERNS:
            if re.search(pattern, text, re.IGNORECASE):
                patterns_found += 1
        
        pattern_ratio = patterns_found / len(self.REQUIRED_PATTERNS)
        confidence_score *= pattern_ratio
        
        if pattern_ratio < 0.5:
            errors.append("Missing required Q&A structure")
            
        # Determine status
        if confidence_score >= 0.8:
            status = ValidationStatus.VALID
        elif confidence_score >= 0.5:
            status = ValidationStatus.CORRECTED
        else:
            status = ValidationStatus.INVALID
            
        return ValidationResult(
            status=status,
            original_text=text,
            cleaned_text=text,  # Could add cleaning logic here
            confidence_score=confidence_score,
            error_messages=errors,
            metadata={'pattern_ratio': pattern_ratio}
        )


def analyze_distribution(records: List[Dict]) -> Dict[str, Any]:
    """Analyze answer distribution in dataset."""
    if not records:
        return {'yes_count': 0, 'no_count': 0, 'yes_percent': 0, 'no_percent': 0}
    
    yes_count = sum(1 for r in records if r['answer'].lower() == 'yes')
    no_count = len(records) - yes_count
    
    return {
        'yes_count': yes_count,
        'no_count': no_count,
        'yes_percent': (yes_count / len(records)) * 100,
        'no_percent': (no_count / len(records)) * 100
    }


def balance_dataset(records: List[Dict], target_ratio: float = 0.5, max_samples: Optional[int] = None) -> List[Dict]:
    """Balance dataset to achieve target yes/no ratio."""
    if not records:
        return records
    
    # Separate by answer
    yes_records = [r for r in records if r['answer'].lower() == 'yes']
    no_records = [r for r in records if r['answer'].lower() == 'no']
    
    # Calculate target counts
    total_available = len(yes_records) + len(no_records)
    if max_samples:
        total_target = min(max_samples, total_available)
    else:
        total_target = total_available
    
    target_yes = int(total_target * target_ratio)
    target_no = total_target - target_yes
    
    # Sample to targets
    actual_yes = min(target_yes, len(yes_records))
    actual_no = min(target_no, len(no_records))
    
    # Randomly sample
    random.shuffle(yes_records)
    random.shuffle(no_records)
    
    balanced_records = yes_records[:actual_yes] + no_records[:actual_no]
    random.shuffle(balanced_records)
    
    return balanced_records


# Load and process teacher data with Q&A-CoT validation
print("🔍 Loading teacher data for Q&A-CoT validation...")

# Load the teacher data
with open(COT_PATH_QA_COT, 'r') as f:
    teacher_data = [json.loads(line) for line in f]

print(f"📊 Loaded {len(teacher_data)} teacher responses")

# Load ground truth
ground_truth_map = {}
with open(SAMPLE_TEST_PATH, 'r') as f:
    for line in f:
        item = json.loads(line)
        ground_truth_map[item['question']] = item['answer']

print(f"🎯 Loaded {len(ground_truth_map)} ground truth answers")

# Initialize and run professional validation pipeline
pipeline = ResponseValidationPipeline()
baseline_records, cot_records, validation_stats = pipeline.process_responses(teacher_data, ground_truth_map)

# Display comprehensive validation results
print(f"\n=== Q&A-COT VALIDATION RESULTS ===")
print(f"📊 Total processed: {validation_stats['total_processed']}")
print(f"✅ Data kept: {validation_stats['data_kept']} ({validation_stats['data_kept']/validation_stats['total_processed']*100:.1f}%)")
print(f"🎯 Average confidence: {validation_stats['avg_confidence']:.3f}")

print(f"\n=== FORMAT PROCESSING RESULTS ===")
print(f"🧠 Q&A-CoT format processed: {validation_stats['qa_cot_format']} ({validation_stats['qa_cot_format']/validation_stats['total_processed']*100:.1f}%)")
print(f"📝 Traditional format processed: {validation_stats['traditional_format']} ({validation_stats['traditional_format']/validation_stats['total_processed']*100:.1f}%)")
print(f"📋 Template fallbacks used: {validation_stats.get('template_fallbacks', 0)} ({validation_stats.get('template_fallbacks', 0)/validation_stats['total_processed']*100:.1f}%)")

print(f"\n=== CONFIDENCE TIER DISTRIBUTION ===")
total_validated = validation_stats['high_confidence'] + validation_stats['medium_confidence'] + validation_stats['low_confidence']
if total_validated > 0:
    print(f"🔥 High (≥0.8): {validation_stats['high_confidence']} ({validation_stats['high_confidence']/total_validated*100:.1f}%)")
    print(f"🟡 Medium (0.5-0.8): {validation_stats['medium_confidence']} ({validation_stats['medium_confidence']/total_validated*100:.1f}%)")
    print(f"🔴 Low (<0.5): {validation_stats['low_confidence']} ({validation_stats['low_confidence']/total_validated*100:.1f}%)")

print(f"\n=== ANSWER VALIDATION ===")
print(f"✅ Valid teacher answers: {validation_stats['valid_teacher_answers']}")
print(f"❌ Invalid teacher answers: {validation_stats['invalid_teacher_answers']}")
if 'teacher_agreement_rate' in validation_stats:
    print(f"🤝 Teacher-ground truth agreement: {validation_stats['teacher_agreement_rate']:.1f}%")

print(f"\n=== ERROR ANALYSIS ===")
if validation_stats['validation_errors']:
    print("Most common validation errors:")
    for error, count in validation_stats['validation_errors'].most_common(3):
        print(f"  - {error}: {count} occurrences")

print(f"🔧 Correction attempts: {validation_stats['correction_attempts']}")
print(f"✅ Successful corrections: {validation_stats['successful_corrections']}")

# Analyze class distributions before balancing
print(f"\n=== DISTRIBUTION ANALYSIS (PRE-BALANCING) ===")
baseline_dist = analyze_distribution(baseline_records)
cot_dist = analyze_distribution(cot_records)

print(f"Baseline: Yes={baseline_dist['yes_count']} ({baseline_dist['yes_percent']:.1f}%), No={baseline_dist['no_count']} ({baseline_dist['no_percent']:.1f}%)")
print(f"Q&A-CoT: Yes={cot_dist['yes_count']} ({cot_dist['yes_percent']:.1f}%), No={cot_dist['no_count']} ({cot_dist['no_percent']:.1f}%)")

# Apply intelligent class balancing
TARGET_TRAIN_SIZE = 1500  # Conservative target for quality
balanced_baseline_records = balance_dataset(baseline_records, target_ratio=0.5, max_samples=TARGET_TRAIN_SIZE)
balanced_cot_records = balance_dataset(cot_records, target_ratio=0.5, max_samples=TARGET_TRAIN_SIZE)

# Analyze post-balancing distributions
balanced_baseline_dist = analyze_distribution(balanced_baseline_records)
balanced_cot_dist = analyze_distribution(balanced_cot_records)

print(f"\n=== DISTRIBUTION ANALYSIS (POST-BALANCING) ===")
print(f"Baseline: Yes={balanced_baseline_dist['yes_count']} ({balanced_baseline_dist['yes_percent']:.1f}%), No={balanced_baseline_dist['no_count']} ({balanced_baseline_dist['no_percent']:.1f}%)")
print(f"Q&A-CoT: Yes={balanced_cot_dist['yes_count']} ({balanced_cot_dist['yes_percent']:.1f}%), No={balanced_cot_dist['no_count']} ({balanced_cot_dist['no_percent']:.1f}%)")

# Save balanced training datasets  
print(f"\n🚀 Saving balanced Q&A-CoT enhanced training datasets...")

# Save baseline dataset (direct Q→A)
with open(BASELINE_PATH, 'w') as f:
    for record in balanced_baseline_records:
        f.write(json.dumps(record) + '\n')

# Save Q&A-CoT enhanced dataset
with open(COT_PATH, 'w') as f:
    for record in balanced_cot_records:
        f.write(json.dumps(record) + '\n')

print(f"💾 Saved {len(balanced_baseline_records)} baseline records to: {BASELINE_PATH}")
print(f"🧠 Saved {len(balanced_cot_records)} Q&A-CoT records to: {COT_PATH}")

print(f"\n=== Q&A-COT TRAINING DATA GENERATION SUMMARY ===")
print(f"🚀 Pipeline Status: ✅ COMPLETE")
print(f"🧠 Format Innovation: Q&A-CoT interleaved self-questioning")
print(f"📊 Data Quality: {validation_stats['data_kept']} high/medium confidence examples")
print(f"⚖️  Class Balance: ~{balanced_baseline_dist['yes_percent']:.0f}% Yes, ~{balanced_baseline_dist['no_percent']:.0f}% No")
print(f"🎯 Confidence Score: {validation_stats['avg_confidence']:.3f} average")
print(f"🔍 Validation Rate: {(validation_stats['data_kept']/validation_stats['total_processed']*100):.1f}% data retention")

# Calculate Q&A-CoT adoption rate
qa_cot_adoption = validation_stats['qa_cot_format'] / validation_stats['total_processed'] * 100
print(f"🧠 Q&A-CoT Adoption: {qa_cot_adoption:.1f}% of training data uses interleaved Q&A format")

if validation_stats['avg_confidence'] >= 0.7:
    print("🎯 SUCCESS: Achieved high-confidence Q&A-CoT training pipeline")
else:
    print("⚠️  Moderate confidence - consider adjusting thresholds")

if qa_cot_adoption >= 80:
    print("🎯 SUCCESS: High Q&A-CoT format adoption (80%+)")
else:
    print(f"⚠️  Q&A-CoT adoption at {qa_cot_adoption:.1f}% - consider improving prompt consistency")

print(f"\n🏁 Ready for enhanced Phase B training with Q&A-CoT supervision!")

# Update paths for Phase B to use Q&A-CoT enhanced data
BASELINE_PATH_ENHANCED = BASELINE_PATH
COT_PATH_ENHANCED = COT_PATH

print(f"\n📂 Updated training paths:")
print(f"- BASELINE_PATH_ENHANCED = '{BASELINE_PATH_ENHANCED}'")
print(f"- COT_PATH_ENHANCED = '{COT_PATH_ENHANCED}'")

🔍 Loading teacher data for Q&A-CoT validation...
📊 Loaded 200 teacher responses
🎯 Loaded 200 ground truth answers
Processed 100 responses...
Processed 200 responses...

=== Q&A-COT VALIDATION RESULTS ===
📊 Total processed: 200
✅ Data kept: 200 (100.0%)
🎯 Average confidence: 1.000

=== FORMAT PROCESSING RESULTS ===
🧠 Q&A-CoT format processed: 200 (100.0%)
📝 Traditional format processed: 0 (0.0%)
📋 Template fallbacks used: 0 (0.0%)

=== CONFIDENCE TIER DISTRIBUTION ===
🔥 High (≥0.8): 200 (100.0%)
🟡 Medium (0.5-0.8): 0 (0.0%)
🔴 Low (<0.5): 0 (0.0%)

=== ANSWER VALIDATION ===
✅ Valid teacher answers: 200
❌ Invalid teacher answers: 0
🤝 Teacher-ground truth agreement: 76.5%

=== ERROR ANALYSIS ===
🔧 Correction attempts: 0
✅ Successful corrections: 0

=== DISTRIBUTION ANALYSIS (PRE-BALANCING) ===
Baseline: Yes=74 (37.0%), No=126 (63.0%)
Q&A-CoT: Yes=74 (37.0%), No=126 (63.0%)

=== DISTRIBUTION ANALYSIS (POST-BALANCING) ===
Baseline: Yes=74 (42.5%), No=100 (57.5%)
Q&A-CoT: Yes=74 (42.5%), No=1

## Phase A: Baseline Training

Phase A trains the student model on direct question-answer pairs without CoT reasoning. This establishes a baseline performance before implementing self-improvement learning.

In [36]:
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    TrainingArguments, Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import load_dataset
import torch
import os
import gc

# Phase A Configuration
PHASE_A_CONFIG = {
    'model_name': MODEL_NAME,
    'train_file': os.path.join(parent_dir, BASELINE_PATH),
    'output_dir': os.path.join(parent_dir, 'models',
'baseline_phaseA'),
    'max_length': MAX_SEQ_LENGTH,
    'num_epochs': 3,
    'batch_size': 4,
    'gradient_accumulation_steps': 8,
    'learning_rate': 2e-4,
    'use_4bit': USE_4BIT
}

print("=== Phase A: Baseline Training ===")
print(f"Model: {PHASE_A_CONFIG['model_name']}")
print(f"Training file: {PHASE_A_CONFIG['train_file']}")
print(f"Output directory: {PHASE_A_CONFIG['output_dir']}")
print(f"Max sequence length: {PHASE_A_CONFIG['max_length']}")
print(f"Training for {PHASE_A_CONFIG['num_epochs']} epochs")

# Clear previous model from memory
if 'model' in locals():
    del model
if 'tokenizer' in locals():
    del tokenizer
gc.collect()
torch.cuda.empty_cache()

# Load fresh model for training
print(f"Loading model for training: {PHASE_A_CONFIG['model_name']}")

if PHASE_A_CONFIG['use_4bit']:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16
    )
    model = AutoModelForCausalLM.from_pretrained(
        PHASE_A_CONFIG['model_name'],
        quantization_config=bnb_config,
        device_map="auto",
        torch_dtype=torch.float16
    )
    # Prepare model for k-bit training first
    model = prepare_model_for_kbit_training(model)
    print("Model prepared for 4-bit training")
else:
    model = AutoModelForCausalLM.from_pretrained(
        PHASE_A_CONFIG['model_name'],
        torch_dtype=torch.float16
    ).to(device)

tokenizer = AutoTokenizer.from_pretrained(PHASE_A_CONFIG['model_name'])
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# LoRA Configuration - Compatible with 4-bit quantization
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],  # Phi-3.5 specific modules
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
    inference_mode=False,
)

try:
    model = get_peft_model(model, lora_config)
    model.print_trainable_parameters()
    print("✅ LoRA adapter successfully attached")
except Exception as e:
    print(f"❌ Error applying LoRA: {e}")
    print("Troubleshooting: Using alternative LoRA configuration...")

    # Alternative LoRA config for compatibility
    lora_config = LoraConfig(
        r=8,  # Smaller rank
        lora_alpha=16,
        target_modules="all-linear",  # Auto-detect linear layers
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM"
    )
    model = get_peft_model(model, lora_config)
    model.print_trainable_parameters()
    print("✅ Alternative LoRA configuration applied successfully")

print(f"GPU Memory after model setup: {torch.cuda.memory_allocated()/1e9:.2f} GB")

=== Phase A: Baseline Training ===
Model: microsoft/Phi-3.5-mini-instruct
Training file: c:\Users\noham\Desktop\Self-Improving-LLM\data\train\train_baseline.jsonl
Output directory: c:\Users\noham\Desktop\Self-Improving-LLM\models\baseline_phaseA
Max sequence length: 2048
Training for 3 epochs
Loading model for training: microsoft/Phi-3.5-mini-instruct


2025-08-19 19:50:46,788 - INFO - We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).
Loading checkpoint shards: 100%|██████████| 2/2 [00:14<00:00,  7.32s/it]


Model prepared for 4-bit training
trainable params: 8,912,896 || all params: 3,829,992,448 || trainable%: 0.2327
✅ LoRA adapter successfully attached
GPU Memory after model setup: 4.96 GB


## Phase A Evaluation

Now we evaluate the baseline model on the test set to establish our performance baseline. According to the training plan, we expect around 60% accuracy from the baseline model.

**What this evaluation does:**
- Loads the StrategyQA test set (687 examples)
- Generates Yes/No answers using the trained baseline model
- Compares predictions against ground truth labels
- Calculates accuracy and saves results for comparison with future phases

This baseline accuracy is crucial for measuring the effectiveness of Phase B (CoT distillation) and Phase C (DPO alignment), which should achieve +7-10pp and +10pp improvements respectively.

In [ ]:
import json
import torch
from datasets import load_dataset
from tqdm import tqdm
import re

# Fix device definition
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def evaluate_model_on_test(model, tokenizer, test_file, device):
    """Evaluate the model on test set and return accuracy."""

    # Load test data
    test_dataset = load_dataset('json', data_files=test_file, split='train')

    model.eval()
    correct = 0
    total = 0

    print(f"Evaluating on {len(test_dataset)} test examples...")

    for example in tqdm(test_dataset, desc="Evaluating"):
        question = example['question']
        ground_truth = "Yes" if example['answer'] else "No"

        # Format prompt same as training
        prompt = f"Question: {question}\nAnswer:"

        # Tokenize
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(device)

        # Generate
        with torch.no_grad():
            outputs = model.generate(
                inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                max_new_tokens=QUICK_EVAL_MAX_TOKENS,
                do_sample=False,
                pad_token_id=tokenizer.pad_token_id
            )

        # Extract generated answer
        generated_text = tokenizer.decode(outputs[0][len(inputs['input_ids'][0]):], skip_special_tokens=True)

        # Parse Yes/No answer
        predicted_answer = "No"  # Default
        if re.search(r'\byes\b', generated_text.lower()):
            predicted_answer = "Yes"
        elif re.search(r'\bno\b', generated_text.lower()):
            predicted_answer = "No"

        # Check correctness
        if predicted_answer == ground_truth:
            correct += 1
        total += 1

        # Debug first few examples
        if total <= 3:
            print(f"Q: {question[:50]}...")
            print(f"Generated: '{generated_text.strip()}'")
            print(f"Predicted: {predicted_answer}, Ground truth: {ground_truth}")
            print("---")

    accuracy = correct / total * 100
    print(f"\n=== Phase A Baseline Results ===")
    print(f"Test Accuracy: {accuracy:.1f}% ({correct}/{total})")
    print(f"Target was ~60%, {'✅ SUCCESS' if accuracy >= 55 else '⚠️ BELOW TARGET'}")

    return accuracy

# Run evaluation on test set
parent_dir = os.path.dirname(os.getcwd())
test_file = os.path.join(parent_dir, 'data', 'raw', 'strategyqa_test.jsonl')

baseline_accuracy = evaluate_model_on_test(model, tokenizer, test_file, device)

# Save results for comparison with later phases
results = {
    'phase_a_baseline_accuracy': baseline_accuracy,
    'model_path': PHASE_A_CONFIG['output_dir'],
    'dataset_size': len(tokenized_dataset),
    'training_epochs': PHASE_A_CONFIG['num_epochs']
}

results_file = os.path.join(parent_dir, 'results_phase_a.json')
with open(results_file, 'w') as f:
    json.dump(results, f, indent=2)

print(f"Results saved to: {results_file}")

## Phase B: CoT Distillation Training

Phase B trains the student model on Chain-of-Thought (CoT) data, incorporating teacher reasoning to improve performance. This builds on the Phase A baseline by teaching the model to benefit from structured reasoning context.

**Approach**: Start from Phase A checkpoint and train on CoT data format:
```
Question: {question}
Student draft: {student_draft}
Teacher reasoning: {teacher_reasoning}
Answer: {answer}
```

**Target**: Achieve 67-70% accuracy (+7-10pp improvement over Phase A baseline of 59.4%)

In [ ]:
import torch
import gc
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    TrainingArguments, Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel
from datasets import load_dataset
import os
import json

# Phase B Configuration - UPDATED TO USE FIXED DATA
PHASE_B_CONFIG = {
    'base_checkpoint': PHASE_A_CONFIG['output_dir'],  # Start from Phase A
    'train_file': os.path.join(parent_dir, COT_PATH_FIXED),  # Use FIXED CoT data
    'output_dir': os.path.join(parent_dir, 'models', 'cot_phaseB_fixed'),  # New output dir
    'max_length': 2048,
    'num_epochs': 3,
    'batch_size': 2,  # Same as Phase A
    'gradient_accumulation_steps': 16,
    'learning_rate': 2e-4,  # Same as Phase A
    'use_4bit': USE_4BIT
}

print("=== Phase B: CoT Distillation Training (FIXED VERSION) ===")
print(f"🔧 Using FIXED datasets with all improvements applied")
print(f"Base checkpoint: {PHASE_B_CONFIG['base_checkpoint']}")
print(f"CoT training file: {PHASE_B_CONFIG['train_file']}")
print(f"Output directory: {PHASE_B_CONFIG['output_dir']}")
print(f"Max sequence length: {PHASE_B_CONFIG['max_length']}")
print(f"Training for {PHASE_B_CONFIG['num_epochs']} epochs")

# Clear previous model from memory
if 'model' in locals():
    del model
if 'tokenizer' in locals():
    del tokenizer
gc.collect()
torch.cuda.empty_cache()

# Load base model fresh
print(f"Loading base model: {MODEL_NAME}")
if PHASE_B_CONFIG['use_4bit']:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16
    )
    base_model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        quantization_config=bnb_config,
        device_map="auto",
        torch_dtype=torch.float16
    )
    base_model = prepare_model_for_kbit_training(base_model)
else:
    base_model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        torch_dtype=torch.float16
    ).to(device)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Load Phase A LoRA adapters
print(f"Loading Phase A checkpoint from: {PHASE_B_CONFIG['base_checkpoint']}")
model = PeftModel.from_pretrained(base_model, PHASE_B_CONFIG['base_checkpoint'])

print("✅ Phase A checkpoint loaded successfully")
print(f"GPU Memory after loading checkpoint: {torch.cuda.memory_allocated()/1e9:.2f} GB")

# Load and prepare FIXED CoT training data
print("Loading FIXED CoT training data...")
print(f"📊 Expected improvements: Balanced classes, more data, consistent format")
cot_train_dataset = load_dataset('json', data_files=PHASE_B_CONFIG['train_file'], split='train')

def cot_tokenize_function(examples):
    # CoT format: Already simplified and consistent with evaluation
    texts = []
    for prompt, answer in zip(examples['prompt'], examples['answer']):
        # The prompt is already in the correct simplified format
        formatted_text = f"{prompt}\nAnswer: {answer}"
        texts.append(formatted_text)

    tokenized = tokenizer(
        texts,
        truncation=True,
        padding='max_length',
        max_length=PHASE_B_CONFIG['max_length'],
        return_tensors=None
    )

    # For causal LM, labels are the same as input_ids
    tokenized['labels'] = tokenized['input_ids'].copy()
    return tokenized

# Tokenize the CoT dataset
print("Tokenizing FIXED CoT dataset...")
cot_tokenized_dataset = cot_train_dataset.map(cot_tokenize_function, batched=True, remove_columns=cot_train_dataset.column_names)

# Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

print(f"FIXED CoT training dataset size: {len(cot_tokenized_dataset)}")
print(f"📈 Expected improvement over original: {len(cot_tokenized_dataset)/137:.1f}x more training data")

# Training arguments for Phase B - Optimized for better results
training_args = TrainingArguments(
    output_dir=PHASE_B_CONFIG['output_dir'],
    overwrite_output_dir=True,
    num_train_epochs=PHASE_B_CONFIG['num_epochs'],
    per_device_train_batch_size=PHASE_B_CONFIG['batch_size'],
    gradient_accumulation_steps=PHASE_B_CONFIG['gradient_accumulation_steps'],
    learning_rate=PHASE_B_CONFIG['learning_rate'],
    warmup_steps=10,  # Increased warmup for stability
    logging_steps=5,
    save_steps=100,  # Save more frequently
    save_total_limit=3,  # Keep more checkpoints
    prediction_loss_only=True,
    remove_unused_columns=False,
    dataloader_pin_memory=False,
    # Optimized precision settings
    fp16=False,  # Disable FP16 to avoid inf checks error
    bf16=torch.cuda.is_bf16_supported(),  # Use BF16 if available
    dataloader_num_workers=0,
    report_to=None,
    # Memory and stability optimizations
    gradient_checkpointing=True,
    dataloader_persistent_workers=False,
    skip_memory_metrics=True,
    # Evaluation and early stopping
    eval_strategy="no",  # No eval set for now
    load_best_model_at_end=False,
)

# Create and run trainer for Phase B
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=cot_tokenized_dataset,
    data_collator=data_collator,
)

print("🚀 Starting Phase B CoT distillation training with FIXED data...")
print("🔧 Improvements applied:")
print("  ✅ Balanced classes (~45% Yes, ~55% No)")
print("  ✅ More training data (all valid teacher responses)")
print("  ✅ Consistent format (matches evaluation)")
print("  ✅ Optimized training parameters")

try:
    trainer.train()
    trainer.save_model()
    print("✅ Phase B training completed successfully!")
    training_success = True
except Exception as e:
    print(f"❌ Training error: {e}")
    print("🔄 Attempting fallback training configuration...")

    # Fallback: More conservative settings
    training_args_fallback = TrainingArguments(
        output_dir=PHASE_B_CONFIG['output_dir'],
        overwrite_output_dir=True,
        num_train_epochs=PHASE_B_CONFIG['num_epochs'],
        per_device_train_batch_size=1,  # Smaller batch
        gradient_accumulation_steps=32,  # Compensate with more accumulation
        learning_rate=1e-4,  # Lower learning rate
        warmup_steps=5,
        logging_steps=10,
        save_steps=100,
        save_total_limit=2,
        prediction_loss_only=True,
        remove_unused_columns=False,
        dataloader_pin_memory=False,
        fp16=False,
        bf16=False,  # Disable all precision optimizations
        dataloader_num_workers=0,
        report_to=None,
        gradient_checkpointing=False,  # Disable gradient checkpointing
    )

    trainer_fallback = Trainer(
        model=model,
        args=training_args_fallback,
        train_dataset=cot_tokenized_dataset,
        data_collator=data_collator,
    )

    print("🔄 Retrying with fallback configuration...")
    trainer_fallback.train()
    trainer_fallback.save_model()
    print("✅ Phase B training completed with fallback configuration!")
    training_success = True

if training_success:
    print(f"🎯 FIXED CoT model saved to: {PHASE_B_CONFIG['output_dir']}")
    print(f"📊 Training completed on {len(cot_tokenized_dataset)} balanced examples")
    print(f"🔬 Ready for evaluation to measure improvements!")

print(f"💾 Final GPU Memory: {torch.cuda.memory_allocated()/1e9:.2f} GB")

## Phase B Evaluation

Now we evaluate the CoT-distilled model to measure the improvement over Phase A baseline. We expect to see +7-10pp accuracy improvement, targeting 67-70% accuracy.

**Evaluation approach:**
1. Load the Phase B CoT model
2. Test on the same StrategyQA test set (687 examples)
3. Compare against Phase A baseline (59.4%)
4. Analyze the improvement from CoT distillation

In [ ]:
import json
import torch
from datasets import load_dataset
from tqdm import tqdm
import re

def evaluate_cot_model_on_test(model, tokenizer, test_file, device, include_reasoning=True):
    """Evaluate the CoT model on test set with optional reasoning context."""

    # Load test data
    test_dataset = load_dataset('json', data_files=test_file, split='train')

    model.eval()
    correct = 0
    total = 0

    print(f"Evaluating CoT model on {len(test_dataset)} test examples...")
    print(f"Include reasoning context: {include_reasoning}")

    for example in tqdm(test_dataset, desc="Evaluating CoT"):
        question = example['question']
        ground_truth = "Yes" if example['answer'] else "No"

        if include_reasoning:
            # For CoT evaluation, we need to provide some reasoning context
            # Since we don't have real student drafts for test set, create a simple format
            prompt = f"Question: {question}\nStudent draft: Answer: Uncertain - need to analyze this carefully\nQuestions: What are the key factors? What evidence supports each side?\nTeacher reasoning: Let me think through this step by step.\n\nAnswer:"
        else:
            # Baseline format for comparison
            prompt = f"Question: {question}\nAnswer:"

        # Tokenize (limit to reasonable length due to longer CoT prompts)
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to(device)

        # Generate
        with torch.no_grad():
            outputs = model.generate(
                inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                max_new_tokens=QUICK_EVAL_MAX_TOKENS,
                do_sample=False,
                pad_token_id=tokenizer.pad_token_id
            )

        # Extract generated answer
        generated_text = tokenizer.decode(outputs[0][len(inputs['input_ids'][0]):], skip_special_tokens=True)

        # Parse Yes/No answer
        predicted_answer = "No"  # Default
        if re.search(r'\\byes\\b', generated_text.lower()):
            predicted_answer = "Yes"
        elif re.search(r'\\bno\\b', generated_text.lower()):
            predicted_answer = "No"

        # Check correctness
        if predicted_answer == ground_truth:
            correct += 1
        total += 1

        # Debug first few examples
        if total <= 3:
            print(f"Q: {question[:50]}...")
            print(f"Generated: '{generated_text.strip()}'")
            print(f"Predicted: {predicted_answer}, Ground truth: {ground_truth}")
            print("---")

    accuracy = correct / total * 100
    return accuracy, correct, total

# Run Phase B evaluation
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
test_file = os.path.join(parent_dir, 'data', 'raw', 'strategyqa_test.jsonl')

print("=== Phase B CoT Model Evaluation ===")

# Evaluate CoT model with reasoning context
cot_accuracy, cot_correct, cot_total = evaluate_cot_model_on_test(
    model, tokenizer, test_file, device, include_reasoning=True
)

print(f"\\n=== Phase B CoT Results ===")
print(f"CoT Model Accuracy: {cot_accuracy:.1f}% ({cot_correct}/{cot_total})")
print(f"Phase A Baseline: 59.4% (408/687)")
print(f"Improvement: {cot_accuracy - 59.4:.1f} percentage points")

# Check if we met our target
target_min = 67.0  # +7pp over baseline
target_max = 70.0  # +10pp over baseline
if cot_accuracy >= target_min:
    status = "✅ TARGET ACHIEVED"
elif cot_accuracy >= 59.4 + 3:  # At least some improvement
    status = "🔄 PARTIAL SUCCESS"
else:
    status = "⚠️ NEEDS INVESTIGATION"

print(f"Target range: 67-70% | Status: {status}")

# Save Phase B results
phase_b_results = {
    'phase_b_cot_accuracy': cot_accuracy,
    'phase_a_baseline_accuracy': 59.4,
    'improvement_pp': cot_accuracy - 59.4,
    'model_path': PHASE_B_CONFIG['output_dir'],
    'dataset_size': len(cot_tokenized_dataset),
    'training_epochs': PHASE_B_CONFIG['num_epochs'],
    'target_achieved': cot_accuracy >= target_min
}

results_file_b = os.path.join(parent_dir, 'results_phase_b.json')
with open(results_file_b, 'w') as f:
    json.dump(phase_b_results, f, indent=2)

print(f"\\nPhase B results saved to: {results_file_b}")

# Optional: Compare CoT vs No-CoT on same model
print("\\n=== Ablation Study ===")
print("Testing same model without reasoning context...")

no_cot_accuracy, no_cot_correct, no_cot_total = evaluate_cot_model_on_test(
    model, tokenizer, test_file, device, include_reasoning=False
)

print(f"Same model without reasoning: {no_cot_accuracy:.1f}% ({no_cot_correct}/{no_cot_total})")
print(f"CoT benefit: {cot_accuracy - no_cot_accuracy:.1f}pp")

# Update results with ablation
phase_b_results['no_cot_accuracy'] = no_cot_accuracy
phase_b_results['cot_benefit'] = cot_accuracy - no_cot_accuracy

with open(results_file_b, 'w') as f:
    json.dump(phase_b_results, f, indent=2)

print("\\n🎯 Phase B evaluation complete!")

# 🎯 TOKEN-LEVEL EMPHASIS ON KEY FACTS AND FINAL ANSWERS

## Overview
This section implements **token-level emphasis** that applies higher attention weights to critical parts of responses during training, specifically targeting key facts and final answers.

## Key Features

### 🎚️ **Emphasis Patterns**
- **Final Answers**: `**Yes**`, `**No**`, `The answer is **X**`
- **Reasoning Markers**: `Question 1:`, `Answer 1:`, `Therefore,`
- **Evidence Indicators**: `Based on`, `key facts`, `need to consider`

### ⚙️ **Technical Implementation**
- **TokenEmphasisDataCollator**: Custom data collator with pattern-based emphasis
- **EmphasisSFTTrainer**: Enhanced SFT trainer with weighted loss computation
- **Adaptive Tracking**: Monitors emphasis effectiveness during training

### 📈 **Emphasis Multipliers**
- **Stage 1**: 2.0x weight on key tokens (light emphasis)
- **Stage 2**: 2.5x weight on key tokens (full emphasis)
- **Configurable**: Adjustable emphasis strength per training stage

## When to Run These Cells

### ⚠️ **EXECUTION ORDER**

**Run these cells AFTER:**
1. ✅ Enhanced Q&A-CoT training data generation
2. ✅ Progressive curriculum dataset preparation

**Run these cells BEFORE:**
- Progressive curriculum training execution
- Enhanced training with token emphasis

### 🔄 **Integration with Progressive Curriculum**

These token emphasis components are **automatically integrated** into the Enhanced Progressive Curriculum Training. The emphasis multipliers are applied progressively:

```
Stage 1: Light Emphasis (2.0x) → Stage 2: Full Emphasis (2.5x)
```

## Expected Benefits

- **Focused Learning**: Higher attention on critical reasoning steps
- **Better Answer Generation**: Enhanced focus on final **Yes**/**No** decisions  
- **Improved Pattern Recognition**: Stronger learning of Q&A-CoT format
- **Adaptive Effectiveness**: Real-time monitoring of emphasis impact

## Files Generated
- `token_emphasis_config.json` - Emphasis patterns and configuration
- Emphasis statistics integrated into training results

---

**💡 Note**: These cells prepare the token emphasis infrastructure. The actual emphasis is applied during the Enhanced Progressive Curriculum Training."

In [ ]:
# ============================================================================
# TOKEN-LEVEL EMPHASIS ON KEY FACTS AND FINAL ANSWERS
# ============================================================================

import torch
import torch.nn.functional as F
from transformers import DataCollatorForLanguageModeling
from typing import List, Dict, Any, Optional
import re
import numpy as np

class TokenEmphasisDataCollator(DataCollatorForLanguageModeling):
    """Data collator that applies emphasis weights to specific token patterns."""
    
    def __init__(self, tokenizer, emphasis_multiplier=2.0, **kwargs):
        super().__init__(tokenizer, **kwargs)
        self.emphasis_multiplier = emphasis_multiplier
        
        # Define patterns that should receive emphasis
        self.emphasis_patterns = [
            r'\*\*(?:Yes|No)\*\*',                    # Bold Yes/No answers
            r'The answer is \*\*(?:Yes|No)\*\*',     # Definitive conclusions
            r'Therefore.*?the answer is',             # Reasoning conclusions
            r'Question \d+:',                         # Question markers
            r'Answer \d+:',                           # Answer markers
            r'Let me think step by step',             # CoT initiators
            r'Let me analyze this',                   # Analysis initiators
            r'Based on.*?analysis',                   # Analysis conclusions
            r'In conclusion',                         # Conclusion markers
            r'Final answer:',                         # Final answer markers
            r'The correct answer is',                 # Correctness assertions
            r'Therefore, the final answer is',        # Final conclusions
            r'Hence,',                               # Logical connectives
            r'Thus,',                                # Logical connectives
        ]
        
    def __call__(self, features):
        # Standard processing
        batch = super().__call__(features)
        
        # Add emphasis weights
        if 'input_ids' in batch:
            emphasis_weights = self._compute_emphasis_weights(batch['input_ids'])
            batch['emphasis_weights'] = emphasis_weights
            
        return batch
    
    def _compute_emphasis_weights(self, input_ids):
        """Compute emphasis weights for the batch."""
        
        batch_size, seq_len = input_ids.shape
        emphasis_weights = torch.ones_like(input_ids, dtype=torch.float)
        
        for batch_idx in range(batch_size):
            # Decode the sequence to text for pattern matching
            tokens = input_ids[batch_idx]
            text = self.tokenizer.decode(tokens, skip_special_tokens=False)
            
            # Find emphasis patterns and mark tokens
            for pattern in self.emphasis_patterns:
                for match in re.finditer(pattern, text, re.IGNORECASE):
                    start_pos, end_pos = match.span()
                    
                    # Find token positions corresponding to the text span
                    start_token_idx = self._find_token_position(text, start_pos, tokens)
                    end_token_idx = self._find_token_position(text, end_pos, tokens)
                    
                    if start_token_idx is not None and end_token_idx is not None:
                        # Apply emphasis to the token range
                        emphasis_weights[batch_idx, start_token_idx:end_token_idx] = self.emphasis_multiplier
        
        return emphasis_weights
    
    def _find_token_position(self, text, char_pos, tokens):
        """Find the token index corresponding to a character position in text."""
        
        # This is a simplified approach - in practice, you might need more sophisticated alignment
        # For now, estimate based on proportional position
        if char_pos >= len(text):
            return len(tokens) - 1
        
        token_ratio = char_pos / len(text)
        token_pos = int(token_ratio * len(tokens))
        return min(token_pos, len(tokens) - 1)

class TokenEmphasisTrainer:
    """Trainer for applying token-level emphasis during loss computation."""
    
    def __init__(self, emphasis_multiplier=2.5, adaptive_emphasis=True):
        self.emphasis_multiplier = emphasis_multiplier
        self.adaptive_emphasis = adaptive_emphasis
        
        # Statistics tracking for adaptive emphasis
        self.emphasis_stats = {
            'total_emphasized_tokens': 0,
            'emphasis_effectiveness': []  # Track how well emphasis works
        }
    
    def create_emphasis_data_collator(self, tokenizer, **kwargs):
        """Create a data collator with emphasis capabilities."""
        return TokenEmphasisDataCollator(
            tokenizer,
            emphasis_multiplier=self.emphasis_multiplier,
            mlm=False,
            return_tensors="pt",
            pad_to_multiple_of=8,
            **kwargs
        )

    def compute_emphasis_loss(self, outputs, labels, emphasis_weights=None):
        """Compute loss with token-level emphasis applied."""

        if emphasis_weights is None:
            # Standard cross-entropy loss
            shift_logits = outputs.logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            loss_fct = torch.nn.CrossEntropyLoss()
            loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
            return loss

        # Apply token-level emphasis
        shift_logits = outputs.logits[..., :-1, :].contiguous()
        shift_labels = labels[..., 1:].contiguous()
        shift_weights = emphasis_weights[..., 1:].contiguous()

        # Compute per-token losses
        loss_fct = torch.nn.CrossEntropyLoss(reduction='none')
        token_losses = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
        token_losses = token_losses.view(shift_labels.shape)

        # Apply emphasis weights
        weighted_losses = token_losses * shift_weights

        # Mask out padding tokens (label = -100)
        mask = (shift_labels != -100).float()
        weighted_losses = weighted_losses * mask

        # Compute final loss
        total_loss = weighted_losses.sum()
        total_tokens = mask.sum()

        if total_tokens > 0:
            loss = total_loss / total_tokens
        else:
            loss = total_loss

        # Track emphasis statistics
        if self.adaptive_emphasis:
            self._update_emphasis_stats(shift_weights, mask)

        return loss

    def _update_emphasis_stats(self, emphasis_weights, mask):
        """Update statistics about emphasis effectiveness."""

        emphasized_tokens = ((emphasis_weights > 1.0) & (mask > 0)).sum().item()
        total_tokens = mask.sum().item()

        self.emphasis_stats['total_emphasized_tokens'] += emphasized_tokens

        if total_tokens > 0:
            emphasis_ratio = emphasized_tokens / total_tokens
            self.emphasis_stats['emphasis_effectiveness'].append(emphasis_ratio)

    def get_emphasis_report(self) -> Dict[str, Any]:
        """Generate a report on emphasis effectiveness."""

        if not self.emphasis_stats['emphasis_effectiveness']:
            return {'status': 'No emphasis data available'}

        effectiveness = self.emphasis_stats['emphasis_effectiveness']

        return {
            'total_emphasized_tokens': self.emphasis_stats['total_emphasized_tokens'],
            'avg_emphasis_ratio': np.mean(effectiveness),
            'emphasis_std': np.std(effectiveness),
            'emphasis_multiplier': self.emphasis_multiplier,
            'batches_processed': len(effectiveness)
        }

# ============================================================================
# ENHANCED TRAINING WITH TOKEN-LEVEL EMPHASIS
# ============================================================================

from transformers import TrainingArguments, Trainer
from trl import SFTTrainer

class EmphasisSFTTrainer(SFTTrainer):
    """SFT Trainer with token-level emphasis support."""

    def __init__(self, emphasis_trainer=None, *args, **kwargs):
        # Filter out parameters not supported by current TRL SFTTrainer
        unsupported_params = ['dataset_text_field', 'max_seq_length']
        filtered_kwargs = {k: v for k, v in kwargs.items() if k not in unsupported_params}
        
        super().__init__(*args, **filtered_kwargs)
        self.emphasis_trainer = emphasis_trainer or TokenEmphasisTrainer()

        # Replace data collator with emphasis-aware version
        if hasattr(self, 'tokenizer'):
            self.data_collator = self.emphasis_trainer.create_emphasis_data_collator(
                self.tokenizer
            )

    def compute_loss(self, model, inputs, return_outputs=False):
        """Override loss computation to apply token emphasis."""

        labels = inputs.get("labels")
        emphasis_weights = inputs.get("emphasis_weights")

        # Forward pass
        outputs = model(**{k: v for k, v in inputs.items() if k not in ['emphasis_weights']})

        # Compute emphasis-aware loss
        if labels is not None:
            loss = self.emphasis_trainer.compute_emphasis_loss(
                outputs, labels, emphasis_weights
            )
        else:
            loss = outputs.loss

        return (loss, outputs) if return_outputs else loss

def analyze_emphasis_patterns(dataset, tokenizer, sample_size=10):
    """Analyze emphasis patterns in the dataset."""

    print("=== ANALYZING TOKEN EMPHASIS PATTERNS ===")

    # Create temporary emphasis data collator for analysis
    emphasis_collator = TokenEmphasisDataCollator(tokenizer, emphasis_multiplier=2.0)

    pattern_stats = {
        'total_samples': 0,
        'samples_with_emphasis': 0,
        'avg_emphasis_tokens_per_sample': 0,
        'pattern_frequencies': {}
    }

    total_emphasis_tokens = 0

    # Analyze a sample of the dataset
    sample_indices = range(min(sample_size, len(dataset)))

    for i in sample_indices:
        example = dataset[i]
        text = example.get('text', '')

        pattern_stats['total_samples'] += 1

        # Check for emphasis patterns
        sample_emphasis_count = 0
        for j, pattern in enumerate(emphasis_collator.emphasis_patterns):
            matches = re.findall(pattern, text, re.IGNORECASE)
            if matches:
                pattern_name = f"pattern_{j+1}"
                if pattern_name not in pattern_stats['pattern_frequencies']:
                    pattern_stats['pattern_frequencies'][pattern_name] = 0
                pattern_stats['pattern_frequencies'][pattern_name] += len(matches)
                sample_emphasis_count += len(matches)

        if sample_emphasis_count > 0:
            pattern_stats['samples_with_emphasis'] += 1

        total_emphasis_tokens += sample_emphasis_count

    # Calculate averages
    if pattern_stats['total_samples'] > 0:
        pattern_stats['avg_emphasis_tokens_per_sample'] = total_emphasis_tokens / pattern_stats['total_samples']
        pattern_stats['emphasis_coverage'] = pattern_stats['samples_with_emphasis'] / pattern_stats['total_samples'] * 100

    # Display results
    print(f"📊 Analyzed {pattern_stats['total_samples']} samples")
    print(f"✅ Samples with emphasis patterns: {pattern_stats['samples_with_emphasis']} ({pattern_stats.get('emphasis_coverage', 0):.1f}%)")
    print(f"🎯 Average emphasis tokens per sample: {pattern_stats['avg_emphasis_tokens_per_sample']:.1f}")

    if pattern_stats['pattern_frequencies']:
        print(f"\n🔍 Pattern frequency breakdown:")
        for pattern, count in pattern_stats['pattern_frequencies'].items():
            print(f"  {pattern}: {count} occurrences")

    return pattern_stats

# ============================================================================
# EXECUTE TOKEN EMPHASIS ENHANCEMENT
# ============================================================================

print("\n🎯 IMPLEMENTING TOKEN-LEVEL EMPHASIS ON KEY FACTS AND FINAL ANSWERS")
print("=======================================================================")

# Initialize token emphasis trainer
emphasis_trainer = TokenEmphasisTrainer(
    emphasis_multiplier=EMPHASIS_MULTIPLIER,  # 2.5x weight for emphasized tokens
    adaptive_emphasis=True
)

print(f"✅ Token emphasis trainer initialized")
print(f"📈 Emphasis multiplier: {emphasis_trainer.emphasis_multiplier}x")
print(f"🔄 Adaptive emphasis: {emphasis_trainer.adaptive_emphasis}")

# Analyze emphasis patterns in our Q&A-CoT dataset
print(f"\n🔍 Analyzing emphasis patterns in curriculum datasets...")

# Analyze Stage 2 dataset (full Q&A-CoT) for emphasis patterns
if 'stage_2_dataset' in locals():
    stage_2_patterns = analyze_emphasis_patterns(stage_2_dataset, tokenizer, sample_size=20)

    print(f"\n=== STAGE 2 Q&A-COT EMPHASIS ANALYSIS ===")
    print(f"Coverage: {stage_2_patterns.get('emphasis_coverage', 0):.1f}% of samples have emphasis patterns")
    print(f"Average emphasis tokens: {stage_2_patterns.get('avg_emphasis_tokens_per_sample', 0):.1f} per sample")

# Create emphasis-aware data collator
emphasis_data_collator = emphasis_trainer.create_emphasis_data_collator(tokenizer)

print(f"\n✅ Token emphasis data collator created")
print(f"🎯 Emphasis patterns configured:")
for i, pattern in enumerate(emphasis_data_collator.emphasis_patterns[:5]):  # Show first 5
    print(f"  {i+1}. {pattern}")
print(f"  ... and {len(emphasis_data_collator.emphasis_patterns)-5} more patterns")

# Test emphasis on a sample
if 'stage_2_dataset' in locals() and len(stage_2_dataset) > 0:
    print(f"\n🧪 Testing token emphasis on sample data...")

    # Get a sample and create a small batch
    sample_data = [stage_2_dataset[0]]

    try:
        # Process through emphasis collator
        test_batch = emphasis_data_collator(sample_data)

        if 'emphasis_weights' in test_batch:
            weights = test_batch['emphasis_weights']
            emphasized_tokens = (weights > 1.0).sum().item()
            total_tokens = weights.numel()

            print(f"✅ Token emphasis test successful!")
            print(f"📊 Emphasized tokens: {emphasized_tokens}/{total_tokens} ({emphasized_tokens/total_tokens*100:.1f}%)")

            # Show weight distribution
            unique_weights = torch.unique(weights)
            print(f"🎚️ Weight distribution: {unique_weights.tolist()}")
        else:
            print(f"⚠️ No emphasis weights generated in test batch")

    except Exception as e:
        print(f"❌ Token emphasis test failed: {e}")

print(f"\n🎯 Token-level emphasis implementation completed!")
print(f"Key features implemented:")
print(f"  ✅ Pattern-based token identification")
print(f"  ✅ Configurable emphasis multipliers")
print(f"  ✅ Adaptive emphasis tracking")
print(f"  ✅ Integration with SFT training")
print(f"  ✅ Emphasis effectiveness monitoring")

# Save emphasis configuration for reference
emphasis_config = {
    'emphasis_multiplier': emphasis_trainer.emphasis_multiplier,
    'adaptive_emphasis': emphasis_trainer.adaptive_emphasis,
    'emphasis_patterns': emphasis_data_collator.emphasis_patterns,
    'implementation_date': '2024-08-16'
}

emphasis_config_file = os.path.join(parent_dir, 'token_emphasis_config.json')
with open(emphasis_config_file, 'w') as f:
    json.dump(emphasis_config, f, indent=2)

print(f"\n📋 Token emphasis configuration saved to: {emphasis_config_file}")
print(f"Ready to integrate with progressive curriculum training!")

# 🎓 PROGRESSIVE CURRICULUM TRAINING FOR Q&A-COT SELF-QUESTIONING

## Overview
This section implements a **two-stage progressive curriculum** for training the model to use Q&A Chain-of-Thought (CoT) self-questioning, as outlined in the StrategyQA Self-Improving LLM implementation plan.

## Training Stages

### Stage 1: Final Reasoning Training
- **Goal**: Teach direct answer generation with basic reasoning
- **Duration**: 1 epoch with higher learning rate (2e-5)
- **Focus**: Learn to conclude with **Yes**/**No** answers
- **Emphasis**: Light token emphasis (2.0x) on key facts

### Stage 2: Full Q&A-CoT Training  
- **Goal**: Learn complete self-questioning format
- **Duration**: 2 epochs with lower learning rate (1e-5)
- **Focus**: Master "Question 1: ... Answer 1: ... Therefore..." format
- **Emphasis**: Full token emphasis (2.5x) on reasoning steps and final answers

## When to Run These Cells

### ⚠️ **IMPORTANT EXECUTION ORDER**

**Run these cells AFTER:**
1. ✅ Basic model loading and LoRA setup
2. ✅ Student draft generation  
3. ✅ Teacher response generation (Q&A-CoT format)
4. ✅ Professional validation pipeline
5. ✅ Enhanced Q&A-CoT training data generation (cell `dec8f3eb`)

**Run these cells BEFORE:**
- Final model evaluation
- Performance comparison with baseline

### 🚀 **Typical Training Workflow**

```
Phase A Baseline → Student Drafts → Teacher Q&A-CoT → Validation → 
PROGRESSIVE CURRICULUM TRAINING → Enhanced Evaluation
```

## Expected Benefits

- **Progressive Learning**: Easier to harder reasoning patterns
- **Token Emphasis**: Higher attention on critical facts and answers  
- **Format Mastery**: Reliable Q&A-CoT self-questioning behavior
- **Improved Performance**: Better than single-stage training approaches

## Files Generated
- `models/progressive_curriculum/stage_1/` - Stage 1 model checkpoint
- `models/enhanced_progressive_curriculum/stage_2_emphasis/` - Final enhanced model
- `curriculum_training_results.json` - Training metrics and statistics
- `enhanced_curriculum_training_results.json` - Enhanced training results with token emphasis

---

**💡 Tip**: These cells implement advanced training techniques. Monitor GPU memory usage and adjust batch sizes if needed."

In [ ]:
# ============================================================================
# PROGRESSIVE CURRICULUM TRAINING FOR Q&A-COT SELF-QUESTIONING
# ============================================================================

import torch
from transformers import TrainingArguments, DataCollatorForLanguageModeling
from trl import SFTTrainer
from datasets import Dataset
import json
import random
from typing import List, Dict, Any
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class ProgressiveCurriculumTrainer:
    """Implements two-stage progressive curriculum for Q&A-CoT training.

    Stage 1: Final reasoning only - teaches direct answer generation
    Stage 2: Full CoT - teaches reasoning + answer generation
    """

    def __init__(self, model, tokenizer):
        """Initialize trainer with model and tokenizer."""
        self.model = model
        self.tokenizer = tokenizer
        
        # Configure training stages
        self.stage_configs = {
            'stage_1': {
                'name': 'Final Reasoning Training',
                'description': 'Train model to generate final answer directly',
                'epochs': int(os.getenv('CURRICULUM_STAGE1_EPOCHS', 1)),
                'learning_rate': float(os.getenv('CURRICULUM_STAGE1_LEARNING_RATE', 5e-5)),
                'warmup_ratio': float(os.getenv('CURRICULUM_STAGE1_WARMUP_RATIO', 0.1)),
                'weight_decay': float(os.getenv('CURRICULUM_STAGE1_WEIGHT_DECAY', 0.01)),
                'batch_size': 4,
                'save_steps': 50,
                'eval_steps': 50,
                'logging_steps': 10
            },
            'stage_2': {
                'name': 'Full Chain-of-Thought Training',
                'description': 'Train model to generate step-by-step reasoning + answer',
                'epochs': int(os.getenv('CURRICULUM_STAGE2_EPOCHS', 2)),
                'learning_rate': float(os.getenv('CURRICULUM_STAGE2_LEARNING_RATE', 3e-5)),
                'warmup_ratio': float(os.getenv('CURRICULUM_STAGE2_WARMUP_RATIO', 0.1)),
                'weight_decay': float(os.getenv('CURRICULUM_STAGE2_WEIGHT_DECAY', 0.01)),
                'batch_size': 4,
                'save_steps': 50,
                'eval_steps': 50,
                'logging_steps': 10
            }
        }

    def create_progressive_datasets(self, student_records: List[Dict]) -> tuple[Dataset, Dataset]:
        """Create stage-specific datasets from student records.
        
        Args:
            student_records: List of dictionaries with 'question', 'reasoning', 'answer'
            
        Returns:
            tuple: (stage_1_dataset, stage_2_dataset)
        """
        
        stage_1_records = []
        stage_2_records = []
        
        print(f"📊 Processing {len(student_records)} student records for progressive curriculum...")
        
        for i, record in enumerate(student_records):
            question = record['question']
            reasoning = record['reasoning']
            answer = record['answer']
            
            # Stage 1: Direct Q->A mapping (final reasoning only)
            # Extract just the final conclusion from reasoning
            final_reasoning = self._extract_final_reasoning(reasoning)
            
            stage_1_prompt = f"Question: {question}\\n\\nAnswer:"
            stage_1_completion = f" {final_reasoning}"
            
            stage_1_records.append({
                'prompt': stage_1_prompt,
                'completion': stage_1_completion,
                'answer': answer,
                'stage': 'stage_1',
                'validation_metadata': {
                    'original_record_id': i,
                    'question': question,
                    'full_reasoning': reasoning
                }
            })
            
            # Stage 2: Full CoT (Q->Reasoning->A)
            stage_2_prompt = f"Question: {question}\\n\\nLet me think through this step by step.\\n\\nAnswer:"
            stage_2_completion = f" {reasoning}"
            
            stage_2_records.append({
                'prompt': stage_2_prompt,
                'completion': stage_2_completion,
                'answer': answer,
                'stage': 'stage_2',
                'validation_metadata': {
                    'original_record_id': i,
                    'question': question,
                    'final_answer_only': final_reasoning
                }
            })
        
        # Convert to datasets
        stage_1_dataset = self._prepare_training_dataset(stage_1_records, "Stage 1")
        stage_2_dataset = self._prepare_training_dataset(stage_2_records, "Stage 2")
        
        print(f"✅ Progressive datasets created:")
        print(f"   📚 Stage 1 (Final Reasoning): {len(stage_1_dataset)} examples")
        print(f"   🧠 Stage 2 (Full CoT): {len(stage_2_dataset)} examples")
        
        return stage_1_dataset, stage_2_dataset

    def _extract_final_reasoning(self, full_reasoning: str) -> str:
        """Extract final reasoning/conclusion from full chain of thought."""
        
        # Split by common conclusion indicators
        conclusion_markers = [
            "therefore",
            "thus",
            "so",
            "hence",
            "in conclusion",
            "finally",
            "the answer is",
            "this means"
        ]
        
        # Try to find the last substantive sentence
        sentences = full_reasoning.split('.')
        
        for sentence in reversed(sentences):
            sentence = sentence.strip()
            if len(sentence) > 10:  # Avoid very short fragments
                # Check if it contains conclusion markers or answer format
                lower_sentence = sentence.lower()
                if any(marker in lower_sentence for marker in conclusion_markers):
                    return sentence
        
        # Fallback: return last substantial sentence
        for sentence in reversed(sentences):
            sentence = sentence.strip()
            if len(sentence) > 15:
                return sentence
        
        # Ultimate fallback: return last 100 characters
        return full_reasoning[-100:].strip()

    def _prepare_training_dataset(self, records: List[Dict], stage_name: str) -> Dataset:
        """Convert records to HuggingFace dataset format."""
        
        # Format for SFT training
        formatted_data = []
        for record in records:
            # Combine prompt and completion for instruction tuning
            full_text = f"{record['prompt']}\\n\\n{record['completion']}"

            formatted_data.append({
                'text': full_text,
                'prompt': record['prompt'],
                'completion': record['completion'],
                'answer': record['answer'],
                'stage': record['stage'],
                'validation_metadata': record.get('validation_metadata', {})
            })

        dataset = Dataset.from_list(formatted_data)
        print(f"✅ {stage_name} dataset prepared: {len(dataset)} examples")

        return dataset

    def _prepare_sft_dataset(self, records: List[Dict], stage_name: str) -> Dataset:
        """Convert records to SFTTrainer-compatible dataset format (text-only)."""
        
        # Format for SFTTrainer - only 'text' field to avoid tensor creation errors
        formatted_data = []
        for record in records:
            # Combine prompt and completion for instruction tuning
            full_text = f"{record['prompt']}\\n\\n{record['completion']}"
            
            # SFTTrainer + TokenEmphasis only need 'text' field
            formatted_data.append({
                'text': full_text
            })
        
        dataset = Dataset.from_list(formatted_data)
        print(f"✅ {stage_name} SFT dataset prepared: {len(dataset)} examples (text-only format)")
        
        return dataset

    def train_progressive_curriculum(self, stage_1_dataset: Dataset, stage_2_dataset: Dataset, output_base_dir: str) -> Dict[str, Any]:
        """Execute the full progressive curriculum training."""

        print("\\n🎓 STARTING PROGRESSIVE CURRICULUM TRAINING")
        print("=====================================================")

        training_results = {}

        # Stage 1: Final Reasoning Training
        print(f"\\n📚 STAGE 1: {self.stage_configs['stage_1']['name']}")
        print(f"Goal: {self.stage_configs['stage_1']['description']}")

        stage_1_model, stage_1_metrics = self._train_stage(
            dataset=stage_1_dataset,
            stage_config=self.stage_configs['stage_1'],
            output_dir=f"{output_base_dir}/stage_1",
            stage_name="stage_1"
        )

        training_results['stage_1'] = stage_1_metrics

        # Stage 2: Full Chain-of-Thought Training
        print(f"\\n🧠 STAGE 2: {self.stage_configs['stage_2']['name']}")
        print(f"Goal: {self.stage_configs['stage_2']['description']}")

        stage_2_model, stage_2_metrics = self._train_stage(
            dataset=stage_2_dataset,
            stage_config=self.stage_configs['stage_2'],
            output_dir=f"{output_base_dir}/stage_2",
            stage_name="stage_2",
            base_model=stage_1_model  # Continue from stage 1
        )

        training_results['stage_2'] = stage_2_metrics

        # Compile final results
        final_results = {
            'curriculum_type': 'progressive_two_stage',
            'final_model': stage_2_model,
            'stage_results': training_results,
            'curriculum_summary': {
                'stage_1_examples': len(stage_1_dataset),
                'stage_2_examples': len(stage_2_dataset),
                'total_training_time': training_results.get('stage_1', {}).get('train_runtime', 0) +
                                     training_results.get('stage_2', {}).get('train_runtime', 0)
            }
        }

        print("\\n✅ PROGRESSIVE CURRICULUM TRAINING COMPLETE!")
        print(f"📈 Final model available with combined Stage 1 + Stage 2 training")
        print(f"⏱️  Total training time: {final_results['curriculum_summary']['total_training_time']:.1f}s")

        return final_results

    def _train_stage(self, dataset: Dataset, stage_config: Dict, output_dir: str, stage_name: str, base_model=None) -> tuple:
        """Train a single curriculum stage."""
        
        # Use base model if provided, otherwise use original model
        model_to_train = base_model if base_model is not None else self.model
        
        # Training arguments for this stage
        training_args = TrainingArguments(
            output_dir=output_dir,
            num_train_epochs=stage_config['epochs'],
            per_device_train_batch_size=4,
            gradient_accumulation_steps=2,
            learning_rate=stage_config['learning_rate'],
            warmup_ratio=stage_config['warmup_ratio'],
            weight_decay=stage_config['weight_decay'],
            logging_steps=10,
            save_strategy="epoch",
            eval_strategy="no",  # No validation for curriculum stages
            fp16=True,
            dataloader_drop_last=False,
            remove_unused_columns=False,
            load_best_model_at_end=False,
            report_to=None,  # Disable wandb/tensorboard
            gradient_checkpointing=True
        )

        # Convert to SFT-compatible format (text-only) to avoid tensor creation errors
        sft_dataset_data = []
        for example in dataset:
            sft_dataset_data.append({'text': example['text']})
        
        sft_dataset = Dataset.from_list(sft_dataset_data)

        # Initialize trainer
        trainer = SFTTrainer(
            model=model_to_train,
            tokenizer=self.tokenizer,
            train_dataset=sft_dataset,
            args=training_args,
            max_seq_length=1024,
            packing=False,  # Don't pack sequences
            dataset_text_field="text"
        )

        print(f"🚀 Training {stage_name}...")
        print(f"   📊 Dataset size: {len(sft_dataset)}")
        print(f"   🎯 Epochs: {stage_config['epochs']}")
        print(f"   📈 Learning rate: {stage_config['learning_rate']}")
        print(f"   🔥 Warmup ratio: {stage_config['warmup_ratio']}")
        print(f"   ⚖️  Weight decay: {stage_config['weight_decay']}")

        # Execute training
        trainer.train()

        # Save the trained model
        trainer.save_model()
        print(f"✅ {stage_name} training complete!")
        print(f"💾 Model saved to: {output_dir}")

        # Return trained model and metrics
        return trainer.model, trainer.state.log_history

# Testing progressive curriculum functionality
if False:  # Set to True to run tests
    print("🧪 Testing Progressive Curriculum Trainer...")
    
    # Note: In real usage, you would initialize with your actual model and tokenizer
    # trainer = ProgressiveCurriculumTrainer(model, tokenizer, "test_model")
    # stage_1_dataset, stage_2_dataset = trainer.create_progressive_datasets(sample_records)
    # results = trainer.train_progressive_curriculum(stage_1_dataset, stage_2_dataset, "test_output")
    
    print("✅ Progressive curriculum trainer ready for use!")

In [34]:
# ============================================================================
# PREPARE PROGRESSIVE CURRICULUM DATASETS
# ============================================================================

print("🎓 PREPARING PROGRESSIVE CURRICULUM DATASETS")
print("=" * 50)

# Load enhanced CoT training data from Build Training Corpora (with reasoning)
enhanced_cot_path = os.path.join(TRAIN_DIR, 'train_cot.jsonl')

# Add error handling for file existence
if not os.path.exists(enhanced_cot_path):
    print(f"❌ Enhanced CoT file not found: {enhanced_cot_path}")
    print("📋 Please run the 'Build Training Corpora' cell first to generate training files")
    raise FileNotFoundError(f"Required training file not found: {enhanced_cot_path}")

print(f"📁 Loading enhanced CoT data from: {enhanced_cot_path}")
with open(enhanced_cot_path, 'r', encoding='utf-8') as f:
    enhanced_cot_data = [json.loads(line) for line in f]

print(f"📊 Loaded {len(enhanced_cot_data)} enhanced CoT records")

# Convert to the format expected by ProgressiveCurriculumTrainer
curriculum_records = []
for record in enhanced_cot_data:
    # Extract the question from the prompt
    question_match = re.search(r'Question:\s*(.+?)(?:\n|$)', record['prompt'])
    question = question_match.group(1).strip() if question_match else "Unknown question"
    
    # Extract the reasoning from the CoT prompt
    # The reasoning is everything after the first question line until "Therefore"
    reasoning_match = re.search(r'Question:\s*[^\n]+\n\s*(.*?)(?:\s*Therefore|$)', record['prompt'], re.DOTALL)
    if reasoning_match:
        reasoning = reasoning_match.group(1).strip()
    else:
        # Fallback: use the entire prompt as reasoning
        reasoning = record['prompt']
    
    curriculum_record = {
        'question': question,
        'reasoning': reasoning,  # Add the required reasoning field
        'prompt': record['prompt'],
        'completion': f"The answer is **{record['answer']}**.",  # Simple completion for curriculum training
        'answer': record['answer'],
        'stage': 'curriculum_training',
        'validation_metadata': record.get('validation_metadata', {})
    }
    curriculum_records.append(curriculum_record)

print(f"✅ Converted {len(curriculum_records)} records for curriculum training")

# Initialize curriculum trainer
curriculum_trainer = ProgressiveCurriculumTrainer(
    model=model,
    tokenizer=tokenizer
    )

# Prepare curriculum datasets from enhanced CoT records
try:
    stage_1_dataset, stage_2_dataset = curriculum_trainer.create_progressive_datasets(curriculum_records)
    
    print(f"📊 Stage 1 dataset: {len(stage_1_dataset)} examples")
    print(f"📊 Stage 2 dataset: {len(stage_2_dataset)} examples")
    
    print("✅ Progressive curriculum datasets created successfully!")
    
except Exception as e:
    print(f"❌ Error creating progressive datasets: {e}")
    import traceback
    traceback.print_exc()

🎓 PREPARING PROGRESSIVE CURRICULUM DATASETS
📁 Loading enhanced CoT data from: c:\Users\noham\Desktop\Self-Improving-LLM\data\train\train_cot.jsonl
📊 Loaded 174 enhanced CoT records
✅ Converted 174 records for curriculum training
📊 Processing 174 student records for progressive curriculum...
✅ Stage 1 dataset prepared: 174 examples
✅ Stage 2 dataset prepared: 174 examples
✅ Progressive datasets created:
   📚 Stage 1 (Final Reasoning): 174 examples
   🧠 Stage 2 (Full CoT): 174 examples
📊 Stage 1 dataset: 174 examples
📊 Stage 2 dataset: 174 examples
✅ Progressive curriculum datasets created successfully!


In [37]:
# ============================================================================
# ENHANCED PROGRESSIVE CURRICULUM WITH TOKEN-LEVEL EMPHASIS
# ============================================================================

class EnhancedProgressiveCurriculumTrainer(ProgressiveCurriculumTrainer):
    """Enhanced curriculum trainer with token-level emphasis integration."""
    
    def __init__(self, model, tokenizer, emphasis_multiplier=EMPHASIS_MULTIPLIER):
        super().__init__(model, tokenizer)
        
        # Initialize token emphasis trainer
        self.emphasis_trainer = TokenEmphasisTrainer(
            emphasis_multiplier=emphasis_multiplier,
            adaptive_emphasis=True
        )
        
        # Update stage configs for emphasis-aware training
        self.stage_configs['stage_1']['emphasis_multiplier'] = emphasis_multiplier * 0.8  # Lower emphasis for stage 1
        self.stage_configs['stage_2']['emphasis_multiplier'] = emphasis_multiplier  # Full emphasis for stage 2
        
        print(f"✅ Enhanced curriculum trainer with token emphasis initialized")
        print(f"📈 Stage 1 emphasis: {self.stage_configs['stage_1']['emphasis_multiplier']}x")
        print(f"📈 Stage 2 emphasis: {self.stage_configs['stage_2']['emphasis_multiplier']}x")
    
    def _train_stage_with_emphasis(self, dataset: Dataset, stage_config: Dict, output_dir: str, stage_name: str) -> tuple:
        """Train a curriculum stage with token-level emphasis."""
        
        # Update emphasis multiplier for this stage
        current_emphasis = stage_config.get('emphasis_multiplier', self.emphasis_trainer.emphasis_multiplier)
        stage_emphasis_trainer = TokenEmphasisTrainer(
            emphasis_multiplier=current_emphasis,
            adaptive_emphasis=True
        )
        
        # Training arguments for this stage
        training_args = TrainingArguments(
            output_dir=output_dir,
            num_train_epochs=stage_config['epochs'],
            per_device_train_batch_size=4,
            gradient_accumulation_steps=2,
            learning_rate=stage_config['learning_rate'],
            warmup_ratio=stage_config['warmup_ratio'],
            weight_decay=stage_config['weight_decay'],
            logging_steps=10,
            save_strategy="epoch",
            eval_strategy="no",  # No validation for curriculum stages
            fp16=True,
            dataloader_drop_last=False,
            remove_unused_columns=False,
            load_best_model_at_end=False,
            report_to=None,  # Disable wandb/tensorboard
            gradient_checkpointing=True
        )
        
        # Convert to SFT-compatible format (text-only) to avoid tensor creation errors
        sft_dataset_data = []
        for item in dataset:
            sft_dataset_data.append({
                'text': item['text']  # Extract only the text field
            })
        sft_dataset = Dataset.from_list(sft_dataset_data)
        print(f"📊 Converted to SFT-compatible dataset with emphasis: {len(sft_dataset)} examples")
        
        # Create emphasis-aware SFT trainer
        trainer = EmphasisSFTTrainer(
            model=self.model,
            args=training_args,
            train_dataset=sft_dataset,
            processing_class=self.tokenizer,
            peft_config=None,
            emphasis_trainer=stage_emphasis_trainer
        )        
        print(f"Training {stage_name} with token emphasis ({current_emphasis}x) on {len(dataset)} examples...")
        
        try:
            # Train the stage with emphasis
            trainer.train()
            
            # Save the model
            trainer.save_model()
            self.tokenizer.save_pretrained(output_dir)
            
            # Get training metrics including emphasis stats
            metrics = {
                'stage': stage_name,
                'epochs': stage_config['epochs'],
                'learning_rate': stage_config['learning_rate'],
                'dataset_size': len(dataset),
                'output_dir': output_dir,
                'emphasis_multiplier': current_emphasis,
                'status': 'completed'
            }
            
            # Add emphasis effectiveness metrics
            emphasis_report = stage_emphasis_trainer.get_emphasis_report()
            metrics.update({f'emphasis_{k}': v for k, v in emphasis_report.items()})
            
            if hasattr(trainer.state, 'log_history') and trainer.state.log_history:
                final_loss = trainer.state.log_history[-1].get('train_loss', 'N/A')
                metrics['final_train_loss'] = final_loss
            
            print(f"✅ {stage_name} training with emphasis completed successfully!")
            print(f"📊 Emphasis effectiveness: {emphasis_report.get('avg_emphasis_ratio', 0):.3f} ratio")
            print(f"🎯 Total emphasized tokens: {emphasis_report.get('total_emphasized_tokens', 0)}")
            print(f"Model saved to: {output_dir}")
            
            return trainer.model, metrics
            
        except Exception as e:
            print(f"❌ Error during {stage_name} training with emphasis: {e}")
            
            # Return original model and error metrics
            error_metrics = {
                'stage': stage_name,
                'status': 'failed',
                'error': str(e),
                'dataset_size': len(dataset),
                'emphasis_multiplier': current_emphasis
            }
            
            return self.model, error_metrics
    
    def train_enhanced_progressive_curriculum(self, stage_1_dataset: Dataset, stage_2_dataset: Dataset, output_base_dir: str) -> Dict[str, Any]:
        """Execute progressive curriculum training with token-level emphasis."""
        
        print("\n🎓 STARTING ENHANCED PROGRESSIVE CURRICULUM TRAINING WITH TOKEN EMPHASIS")
        print("============================================================================")
        
        training_results = {}
        
        # Stage 1: Final Reasoning Training with Light Emphasis
        print(f"\n📚 STAGE 1: {self.stage_configs['stage_1']['name']} (Light Emphasis)")
        print(f"Goal: {self.stage_configs['stage_1']['description']}")
        print(f"Emphasis: {self.stage_configs['stage_1']['emphasis_multiplier']}x weight on key tokens")
        
        stage_1_model, stage_1_metrics = self._train_stage_with_emphasis(
            dataset=stage_1_dataset,
            stage_config=self.stage_configs['stage_1'],
            output_dir=f"{output_base_dir}/stage_1_emphasis",
            stage_name="stage_1_emphasis"
        )
        
        training_results['stage_1_emphasis'] = stage_1_metrics
        
        # Stage 2: Full Q&A-CoT Training with Full Emphasis
        print(f"\n🧠 STAGE 2: {self.stage_configs['stage_2']['name']} (Full Emphasis)")
        print(f"Goal: {self.stage_configs['stage_2']['description']}")
        print(f"Emphasis: {self.stage_configs['stage_2']['emphasis_multiplier']}x weight on key tokens")
        print("Starting from Stage 1 trained model...")
        
        # Use the Stage 1 model as starting point for Stage 2
        self.model = stage_1_model
        
        stage_2_model, stage_2_metrics = self._train_stage_with_emphasis(
            dataset=stage_2_dataset,
            stage_config=self.stage_configs['stage_2'],
            output_dir=f"{output_base_dir}/stage_2_emphasis",
            stage_name="stage_2_emphasis"
        )
        
        training_results['stage_2_emphasis'] = stage_2_metrics
        
        # Final model is the Stage 2 model
        self.model = stage_2_model
        
        # Calculate overall emphasis effectiveness
        total_emphasized_tokens = (
            stage_1_metrics.get('emphasis_total_emphasized_tokens', 0) +
            stage_2_metrics.get('emphasis_total_emphasized_tokens', 0)
        )
        
        training_results['overall_emphasis_stats'] = {
            'total_emphasized_tokens': total_emphasized_tokens,
            'stage_1_emphasis_ratio': stage_1_metrics.get('emphasis_avg_emphasis_ratio', 0),
            'stage_2_emphasis_ratio': stage_2_metrics.get('emphasis_avg_emphasis_ratio', 0)
        }
        
        print("\n🎯 ENHANCED PROGRESSIVE CURRICULUM TRAINING COMPLETE!")
        print(f"📊 Total emphasized tokens across both stages: {total_emphasized_tokens}")
        print("Final model ready for evaluation with token emphasis benefits...")
        
        return training_results

# ============================================================================
# EXECUTE ENHANCED PROGRESSIVE CURRICULUM WITH TOKEN EMPHASIS
# ============================================================================

print("\n🚀 LAUNCHING ENHANCED PROGRESSIVE CURRICULUM WITH TOKEN EMPHASIS")
print("=====================================================================")

# Initialize enhanced curriculum trainer with token emphasis
enhanced_curriculum_trainer = EnhancedProgressiveCurriculumTrainer(
    model=model,
    tokenizer=tokenizer,
    emphasis_multiplier=EMPHASIS_MULTIPLIER
)

# Use existing curriculum datasets
if 'stage_1_dataset' in locals() and 'stage_2_dataset' in locals():
    print(f"\n=== ENHANCED CURRICULUM DATASET SUMMARY ===")
    print(f"📚 Stage 1 (Final Reasoning + Light Emphasis): {len(stage_1_dataset)} examples")
    print(f"🧠 Stage 2 (Full Q&A-CoT + Full Emphasis): {len(stage_2_dataset)} examples")
    
    # Define enhanced output directory
    enhanced_curriculum_output_dir = os.path.join(parent_dir, 'models', 'enhanced_progressive_curriculum')
    os.makedirs(enhanced_curriculum_output_dir, exist_ok=True)
    
    # Execute enhanced progressive curriculum training
    print(f"\n🚀 Starting enhanced progressive curriculum training...")
    print(f"Output directory: {enhanced_curriculum_output_dir}")
    
    enhanced_curriculum_results = enhanced_curriculum_trainer.train_enhanced_progressive_curriculum(
        stage_1_dataset=stage_1_dataset,
        stage_2_dataset=stage_2_dataset,
        output_base_dir=enhanced_curriculum_output_dir
    )
    
    # Display enhanced curriculum training results
    print("\n=== ENHANCED PROGRESSIVE CURRICULUM TRAINING RESULTS ===")
    for stage, metrics in enhanced_curriculum_results.items():
        print(f"\n{stage.upper()} RESULTS:")
        for key, value in metrics.items():
            if isinstance(value, float):
                print(f"  {key}: {value:.4f}")
            else:
                print(f"  {key}: {value}")
    
    # Save enhanced curriculum training results
    enhanced_curriculum_results_file = os.path.join(parent_dir, 'enhanced_curriculum_training_results.json')
    with open(enhanced_curriculum_results_file, 'w') as f:
        json.dump(enhanced_curriculum_results, f, indent=2)
    
    print(f"\n📊 Enhanced curriculum training results saved to: {enhanced_curriculum_results_file}")
    
    # Update model reference to the final enhanced curriculum-trained model
    model = enhanced_curriculum_trainer.model
    
    print("\n🎯 Enhanced progressive curriculum training with token emphasis completed!")
    print("Model now has:")
    print("  ✅ Progressive curriculum learning (Stage 1 → Stage 2)")
    print("  ✅ Token-level emphasis on key facts and final answers")
    print("  ✅ Q&A-CoT self-questioning format")
    print("  ✅ Professional validation and confidence scoring")
    print("Ready for enhanced evaluation!")
    
else:
    print("⚠️ Curriculum datasets not found. Please run the progressive curriculum preparation first.")


🚀 LAUNCHING ENHANCED PROGRESSIVE CURRICULUM WITH TOKEN EMPHASIS
✅ Enhanced curriculum trainer with token emphasis initialized
📈 Stage 1 emphasis: 2.0x
📈 Stage 2 emphasis: 2.5x

=== ENHANCED CURRICULUM DATASET SUMMARY ===
📚 Stage 1 (Final Reasoning + Light Emphasis): 174 examples
🧠 Stage 2 (Full Q&A-CoT + Full Emphasis): 174 examples

🚀 Starting enhanced progressive curriculum training...
Output directory: c:\Users\noham\Desktop\Self-Improving-LLM\models\enhanced_progressive_curriculum

🎓 STARTING ENHANCED PROGRESSIVE CURRICULUM TRAINING WITH TOKEN EMPHASIS

📚 STAGE 1: Final Reasoning Training (Light Emphasis)
Goal: Train model to generate final answer directly
Emphasis: 2.0x weight on key tokens
📊 Converted to SFT-compatible dataset with emphasis: 174 examples


Truncating train dataset: 100%|██████████| 174/174 [00:00<00:00, 9430.27 examples/s]
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


Training stage_1_emphasis with token emphasis (2.0x) on 174 examples...
❌ Error during stage_1_emphasis training with emphasis: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`text` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

🧠 STAGE 2: Full Chain-of-Thought Training (Full Emphasis)
Goal: Train model to generate step-by-step reasoning + answer
Emphasis: 2.5x weight on key tokens
Starting from Stage 1 trained model...
📊 Converted to SFT-compatible dataset with emphasis: 174 examples


Truncating train dataset: 100%|██████████| 174/174 [00:00<00:00, 33757.75 examples/s]
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


Training stage_2_emphasis with token emphasis (2.5x) on 174 examples...
❌ Error during stage_2_emphasis training with emphasis: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`text` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

🎯 ENHANCED PROGRESSIVE CURRICULUM TRAINING COMPLETE!
📊 Total emphasized tokens across both stages: 0
Final model ready for evaluation with token emphasis benefits...

=== ENHANCED PROGRESSIVE CURRICULUM TRAINING RESULTS ===

STAGE_1_EMPHASIS RESULTS:
  stage: stage_1_emphasis
  status: failed
  error: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`text` in this case) have excessive nesting (inputs type `list` where type `int` is expected).
  da